# Analysis of deep mutational scanning of influenza A/Aichi/1968 (H3N2) nucleoprotein in perturbed host proteostasis environments

Anna Ponomarenko, March 2018

## Load required modules

In [1]:
import os
import glob
import sys
import string
import time
import multiprocessing

import pandas as pd

basedir = os.getcwd()
if not os.path.isdir('notebook_figures'):
    !mkdir notebook_figures
if not os.path.isdir('FASTQ_files'):
    !mkdir FASTQ_files
if not os.path.isdir('preferences'):
    !mkdir preferences
    
import matplotlib
print "Using matplotlib version %s" % matplotlib.__version__
matplotlib.use("Pdf")
orange = "#E69F00"
lightblue = "#56B4E9"
green = "#009E73"
yellow = "#F0E442"
darkblue = "#0072B2"
rust =  "#D55E00"
purple =  "#CC79A7"
from matplotlib.ticker import MaxNLocator
from matplotlib.patches import Polygon
from matplotlib import gridspec
import pylab as plt

from IPython.display import Image, display

import dms_tools
print "Using dms_tools version %s" % dms_tools.__version__
import dms_tools.utils
import dms_tools.file_io
import dms_tools.plot

Using matplotlib version 2.0.2
Using dms_tools version 1.2.2


### Define some functions and variables used below by multiple cells

In [8]:
def RunScriptDms(rundir, run_name, script_name, commands, use_sbatch, sbatch_cpus, walltime=None):
    """Runs a ``dmstools`` script.

    *rundir* is the directory in which we run the job. Created if it does
    not exist.

    *run_name* is the name of the run, which should be a string without
    spaces. The input file has this prefix followed by ``_infile.txt``.

    *script_name* is the name of the script that we run.

    *commands* is a list of command-line arguments. These arguments should appear 
    in the order in which one would place them on the command line. The elements in this
    list are joined together by spaces, and given to *script_name*. All elements
    should be strings.

    It is assumed that the script can be run at the command line using::

        script_name infile

    Returns *runfailed*: *True* if run failed, and *False* otherwise.
    """
    print "Running %s for %s in directory %s..." % (script_name, run_name, rundir)
    currdir = os.getcwd()
    if not os.path.isdir(rundir):
        os.mkdir(rundir)
    os.chdir(rundir)
    if (not run_name) or not all([x not in string.whitespace for x in run_name]):
        raise ValueError("Invalid run_name of %s" % run_name)
    commandline = ' '.join(['%s' % command for command in commands])
    if use_sbatch:
        sbatchfile = '%s.sbatch' % run_name # sbatch command file
        jobidfile = 'sbatch_%s_jobid' % run_name # holds sbatch job id
        jobstatusfile = 'sbatch_%s_jobstatus' % run_name # holds sbatch job status
        joberrorsfile = 'sbatch_%s_errors' % run_name # holds sbatch job errors
        sbatch_f = open(sbatchfile, 'w')
        sbatch_f.write('#!/bin/sh\n#SBATCH\n')
        if walltime:
            sbatch_f.write('#PBS -l walltime=%d:00:00\n' % walltime)
        sbatch_f.write('%s %s' % (script_name, commandline))
        sbatch_f.close()
        os.system('sbatch -c %d -e %s %s > %s' % (sbatch_cpus, joberrorsfile, sbatchfile, jobidfile))
        time.sleep(60) # short 1 minute delay
        jobid = int(open(jobidfile).read().split()[-1])
        nslurmfails = 0
        while True:
            time.sleep(60) # delay 1 minute
            returncode = os.system('squeue -j %d > %s' % (jobid, jobstatusfile))
            if returncode != 0:
                nslurmfails += 1
                if nslurmfails > 180: # error if squeue fails at least 180 consecutive times
                    raise ValueError("squeue is continually failing, which means that slurm is not working on your system. Note that although this script has crashed, many of the jobs submitted via slurm may still be running. You'll want to monitor (squeue) or kill them (scancel) -- unfortunately you can't do that until slurm starts working again.")
                continue # we got an error while trying to run squeue
            nslurmfails = 0
            lines = open(jobstatusfile).readlines()
            if len(lines) < 2:
                break # no longer in slurm queue
        errors = open(joberrorsfile).read().strip()
    else:
        errors = os.system('%s %s' % (script_name, commandline))
    os.chdir(currdir)
    if errors:
        print "ERROR running %s for %s in directory %s." % (script_name, run_name, rundir)
        return True
    else:
        print "Successfully completed running %s for %s in directory %s." % (script_name, run_name, rundir)
        return False

    
def RunProcesses(processes, nmultiruns):
    """Runs a list *multiprocessing.Process* processes.

    *processes* is a list of *multiprocessing.Process* objects that
    have not yet been started. If an empty list, then just returns 
    with no action taken.

    *nmultiruns* is an integer >= 1 indicating the number of simultaneous
    processes to run.

    Runs the processes in *processes*, making sure to never have more than
    *nmultiruns* running at a time. If any of the processes fail (return
    an exitcode with a boolean value other than *False*), an exception
    is raised immediately. Otherwise, this function finishes when all
    processes have completed.
    """
    if not processes:
        return
    if not (nmultiruns >= 1 and isinstance(nmultiruns, int)):
        raise ValueError("nmultiruns must be an integer >= 1")
    processes_started = [False] * len(processes)
    processes_running = [False] * len(processes)
    processes_finished = [False] * len(processes)
    while not all(processes_finished):
        if (processes_running.count(True) < nmultiruns) and not all(processes_started):
            i = processes_started.index(False)
            processes[i].start()
            processes_started[i] = True
            processes_running[i] = True
        for i in range(len(processes)):
            if processes_running[i]:
                if not processes[i].is_alive():
                    processes_running[i] = False
                    processes_finished[i] = True
                    if processes[i].exitcode:
                        raise IOError("One of the processes failed to complete.")
        time.sleep(60)

max_cpus = 1
## Do we use sbatch when appropriate? Some simple operations will still
## be run without sbatch even when True
#use_sbatch = False

# Specify reference sequence to map reads to, numbering must follow what is specified by alignspecs
refseq = '%s/Aichi_NP_reference.fasta' % basedir

# Specify the biological replicates and sample names. Sample names are stored as list of 2-tuples or in a dictionary
replicates = ['replicate-1', 'replicate-2', 'replicate-3']
replicate_sample = [('replicate-1', 'wt_plasmid'),('replicate-1', 'mutant_plasmid'),('replicate-1', 'p0_wt'),('replicate-1', 'p0_mutant'),('replicate-1', 'wt_37C'),('replicate-1', 'basal_37C'),('replicate-1', 'HSF1i_37C'), ('replicate-1', 'Hsp90i_37C'),('replicate-1', 'wt_39C'),('replicate-1', 'basal_39C'),('replicate-1', 'HSF1i_39C'), ('replicate-1', 'Hsp90i_39C'),('replicate-2', 'wt_plasmid'),('replicate-2', 'mutant_plasmid'),('replicate-2', 'p0_wt'),('replicate-2', 'p0_mutant'),('replicate-2', 'wt_37C'),('replicate-2', 'basal_37C'),('replicate-2', 'HSF1i_37C'), ('replicate-2', 'Hsp90i_37C'),('replicate-2', 'wt_39C'),('replicate-2', 'basal_39C'),('replicate-2', 'HSF1i_39C'), ('replicate-2', 'Hsp90i_39C'),('replicate-3', 'wt_plasmid'),('replicate-3', 'mutant_plasmid'),('replicate-3', 'p0_wt'),('replicate-3', 'p0_mutant'),('replicate-3', 'wt_37C'),('replicate-3', 'basal_37C'),('replicate-3', 'HSF1i_37C'), ('replicate-3', 'Hsp90i_37C'),('replicate-3', 'wt_39C'),('replicate-3', 'basal_39C'),('replicate-3', 'HSF1i_39C'), ('replicate-3', 'Hsp90i_39C')]

replicate_sample_dict = dict([ (replicate, []) for replicate in replicates])
for replicate, sample in replicate_sample:
    replicate_sample_dict[replicate].append(sample)
        
print("Shared functions and variables are now defined.")

Shared functions and variables are now defined.


### Create reference file

In [3]:
%%writefile $refseq
>gi|392340231|gb|CY121120.1| Influenza A virus (A/Aichi/2/1968(H3N2)) nucleocapsid protein (NP) gene, complete cds
ATGGCGTCCCAAGGCACCAAACGGTCTTATGAACAGATGGAAACTGATGGGGAACGCCAGAATGCAACTGAGATCAGAGCATCCGTCGGGAAGATGATTGATGGAAT
TGGACGATTCTACATCCAAATGTGCACTGAACTTAAACTCAGTGATTATGAGGGGCGACTGATCCAGAAC
AGCTTAACAATAGAGAGAATGGTGCTCTCTGCTTTTGACGAAAGAAGGAATAAATATCTGGAAGAACATC
CCAGCGCGGGGAAGGATCCTAAGAAAACTGGAGGACCCATATACAAGAGAGTAGATAGAAAGTGGATGAG
GGAACTCGTCCTTTATGACAAAGAAGAAATAAGGCGAATCTGGCGCCAAGCCAATAATGGTGATGATGCA
ACAGCTGGTCTGACTCACATGATGATCTGGCATTCCAATTTGAATGATACAACATACCAGAGGACAAGAG
CTCTTGTTCGCACCGGCATGGATCCCAGGATGTGCTCTCTGATGCAGGGTTCGACTCTCCCTAGGAGGTC
TGGAGCTGCAGGCGCTGCAGTCAAAGGAGTTGGGACAATGGTGATGGAGTTGATAAGGATGATCAAACGT
GGGATCAATGATCGGAACTTCTGGAGAGGTGAAAATGGACGAAAAACAAGGAGTGCTTACGAGAGAATGT
GCAACATTCTCAAAGGAAAATTTCAAACAGCTGCACAAAGGGCAATGATGGATCAAGTGAGAGAAAGTCG
GAACCCAGGAAATGCTGAGATCGAAGATCTCATCTTTCTGGCACGGTCTGCACTCATATTGAGAGGGTCA
GTTGCTCACAAATCTTGTCTGCCCGCCTGTGTGTATGGACCTGCCGTAGCCAGTGGCTACGACTTCGAAA
AAGAGGGATACTCTTTAGTGGGAATAGACCCTTTCAAACTGCTTCAAAACAGCCAAGTATACAGCCTAAT
CAGACCGAACGAGAATCCAGCACACAAGAGTCAGCTGGTGTGGATGGCATGCAATTCTGCTGCATTTGAA
GATCTAAGAGTATTAAGCTTCATCAGAGGGACCAAAGTATCCCCAAGGGGGAAACTTTCCACTAGAGGAG
TACAAATTGCTTCAAATGAAAACATGGATGCTATGGAATCAAGTACTCTTGAACTGAGAAGCAGGTACTG
GGCCATAAGAACCAGAAGTGGAGGAAACACTAATCAACAGAGGGCCTCTGCAGGTCAAATCAGTGTGCAA
CCTGCATTTTCTGTGCAAAGAAACCTCCCATTTGACAAACCAACCATCATGGCAGCATTCACTGGGAATA
CAGAGGGAAGAACATCAGACATGAGGGCAGAAATTATAAGGATGATGGAAGGTGCAAAACCAGAAGAAAT
GTCCTTCCAGGGGCGGGGAGTCTTCGAGCTCTCGGACGAAAGGGCAGCGAACCCGATCGTGCCCTCTTTT
GACATGAGTAATGAAGGATCTTATTTCTTCGGAGACAATGCAGAGGAGTACGACAATTAA


Writing /Users/annapon/Documents/NP_DMS_2018/2018_NP_DMS/Aichi_NP_reference.fasta


## Process and analyze deep sequencing data

### Align paired-end reads

In [9]:
def main():
    """Main body of script."""

    # Specify R1 and R2 read lengths and gene range of each subamplicon for dms_barcodedsubamplicons
    alignspecs = '1,252,34,32 253,501,38,30 502,750,30,36 751,1002,32,41 1003,1251,29,34 1252,1497,34,41'

    # Specify range of residues to consider
    sitesrange = ' '.join(['%s' %i for i in range(1,499)]) # range of residues to consider

    # Set R1 and R2 trim lengths for dms_barcodedsubamplicons
    R1trimlength = 200
    R2trimlength = 160

    # Group all barcodes with this many reads or more for dms_summarizealignments
    maxperbarcode = 4
 
    # Directory path for FASTQ files for each replicate and amplicon
    FASTQ_path = '%s/FASTQ_files/' % basedir
    
    print "Found FASTQ_path"

    # Toggle which scripts to run or skip:
    # however, this script is not smart enough to know if the existing output 
    # exists necessary to run the next strip exist - so use with caution!
    run_barcodedsubamplicons = False
    run_summarizealignments = True
    run_inferdiffprefs = False
    run_diffprefs_editsites = False
    run_diffprefs_merge = False
    run_diffprefs_correlate = False
    run_diffprefs_logoplot = False
    #########################################
    #########################################
    # Now run dms_barcodedsubamplicons
    # Each sample is run its own subdirectory
    #########################################
    #########################################
    if run_barcodedsubamplicons:
        print "entering barcodedsubamplicons section of masterscript"

        processes = []

        # Store (r1files, r2files) in dictionary keyed by (replicate, sample)
        fastqfiles = {} 
        for replicate, sample in replicate_sample:
            r1files = sorted(glob.glob('%s/%s/%s/%s_1.fastq' % (FASTQ_path, replicate, sample, sample)))
            r2files = sorted(glob.glob('%s/%s/%s/%s_2.fastq' % (FASTQ_path, replicate, sample, sample)))
            assert r1files and r2files
            fastqfiles[(replicate, sample)] = (r1files, r2files)
            #print replicate, sample, r1files, r2files

            if not os.path.isdir(replicate):
                os.mkdir(replicate)
            if not os.path.isdir("%s/%s" % (replicate, sample)):
                os.mkdir("%s/%s" % (replicate, sample))

        # Make the barcoded alignments for each sample
        for replicate, sample in replicate_sample:
            subdir = '%s/%s' % (replicate, sample)
            alignprefix = '%s_' % sample
            (r1files, r2files) = fastqfiles[(replicate, sample)]
            commands = ['--R1trimlength %s' % R1trimlength, '--R2trimlength %s' % R2trimlength, alignprefix, refseq, ','.join(r1files), ','.join(r2files), alignspecs]

            # Add this makealignments.py call to the list of processes.
            # Use *list* to make a copy of *commands* since lists are mutable on subsequent runs
            # of making replicate-amplicon combinations.
            processes.append(multiprocessing.Process(target=RunScriptDms, args=(subdir, 'barcodedsubamplicons', 'dms_barcodedsubamplicons', list(commands), use_sbatch, 1)))
    

        # run ALL the barcodedsubamplicons processes
        RunProcesses(processes, nmultiruns=max_cpus)

        # end barcodedsubamplicons block of masterscript
    else:
        print "Skipping barcodedsubamplicons section of masterscript"
        
if __name__ == '__main__':
    main() # run the script

Found FASTQ_path
Skipping barcodedsubamplicons section of masterscript


### Summarize alignments

In [10]:
def main():
    """Main body of script."""

    # Group all barcodes with this many reads or more for dms_summarizealignments
    maxperbarcode = 4

    # Directory path for FASTQ files for each replicate and amplicon
    FASTQ_path = '%s/FASTQ_files/' % basedir


    # Run summarizealignments on individual samples
    for replicate, sample in replicate_sample:
        subdir = '%s/%s' % (replicate, sample)
        alignprefix_name = '%s_,%s' % (sample, sample)
        outprefix = 'alignmentsummary_'
        commands = ['--maxperbarcode %s' % maxperbarcode, outprefix, 'barcodedsubamplicons', alignprefix_name]            #print commands
#        RunScriptDms(subdir, 'summarizealignments', 'dms_summarizealignments', commands, False, 1)

    # Run summarizealignments on all samples from a replicate     
    for replicate in replicates:
        alignprefixes_names = []
        subdir = replicate
        outprefix = 'alignmentsummary_'
        for sample in replicate_sample_dict[replicate]:
            if replicate == 'replicate-1' and sample != 'DNA':   # only 7 alignments can be summarized currently
                alignprefix = '%s/%s_' % (sample, sample)
                alignprefixes_names.append("%s,%s" % (alignprefix, sample))
            if replicate != 'replicate-1':
                alignprefix = '%s/%s_' % (sample, sample)
                alignprefixes_names.append("%s,%s" % (alignprefix, sample))
        commands = ['--maxperbarcode %s' % maxperbarcode, outprefix, 'barcodedsubamplicons', ' '.join(alignprefixes_names)]
        print commands
        RunScriptDms(subdir, 'summarizealignments', 'dms_summarizealignments', commands, False, 1)
        
if __name__ == '__main__':
    main() # run the script

['--maxperbarcode 4', 'alignmentsummary_', 'barcodedsubamplicons', 'wt_plasmid/wt_plasmid_,wt_plasmid mutant_plasmid/mutant_plasmid_,mutant_plasmid p0_wt/p0_wt_,p0_wt p0_mutant/p0_mutant_,p0_mutant wt_37C/wt_37C_,wt_37C basal_37C/basal_37C_,basal_37C HSF1i_37C/HSF1i_37C_,HSF1i_37C Hsp90i_37C/Hsp90i_37C_,Hsp90i_37C wt_39C/wt_39C_,wt_39C basal_39C/basal_39C_,basal_39C HSF1i_39C/HSF1i_39C_,HSF1i_39C Hsp90i_39C/Hsp90i_39C_,Hsp90i_39C']
Running dms_summarizealignments for summarizealignments in directory replicate-1...
Successfully completed running dms_summarizealignments for summarizealignments in directory replicate-1.
['--maxperbarcode 4', 'alignmentsummary_', 'barcodedsubamplicons', 'wt_plasmid/wt_plasmid_,wt_plasmid mutant_plasmid/mutant_plasmid_,mutant_plasmid p0_wt/p0_wt_,p0_wt p0_mutant/p0_mutant_,p0_mutant wt_37C/wt_37C_,wt_37C basal_37C/basal_37C_,basal_37C HSF1i_37C/HSF1i_37C_,HSF1i_37C Hsp90i_37C/Hsp90i_37C_,Hsp90i_37C wt_39C/wt_39C_,wt_39C basal_39C/basal_39C_,basal_39C HSF1i_

### Plot mutational and error rates, depth of coverage

In [2]:
from IPython.display import Image, display

filepath = 'replicate-1/alignmentsummary_barcodes.pdf'

png = filepath.rstrip('.pdf') + '.png'
!convert -density 192 -trim $filepath $png
display(Image(png, width=500))

filepath = 'replicate-1/alignmentsummary_depth.pdf'

png = filepath.rstrip('.pdf') + '.png'
!convert -density 192 -trim $filepath $png
display(Image(png, width=500))

filepath = 'replicate-1/alignmentsummary_mutcounts_all.pdf'

png = filepath.rstrip('.pdf') + '.png'
!convert -density 192 -trim $filepath $png
display(Image(png, width=500))

filepath = 'replicate-1/alignmentsummary_mutcounts_multi_nt.pdf'

png = filepath.rstrip('.pdf') + '.png'
!convert -density 192 -trim $filepath $png
display(Image(png, width=500))

filepath = 'replicate-1/alignmentsummary_mutdepth.pdf'

png = filepath.rstrip('.pdf') + '.png'
!convert -density 192 -trim $filepath $png
display(Image(png, width=500))

filepath = 'replicate-1/alignmentsummary_mutfreqs.pdf'

png = filepath.rstrip('.pdf') + '.png'
!convert -density 192 -trim $filepath $png
display(Image(png, width=500))

filepath = 'replicate-1/alignmentsummary_reads.pdf'

png = filepath.rstrip('.pdf') + '.png'
!convert -density 192 -trim $filepath $png
display(Image(png, width=500))

/bin/sh: convert: command not found


/bin/sh: convert: command not found


/bin/sh: convert: command not found


/bin/sh: convert: command not found


/bin/sh: convert: command not found


/bin/sh: convert: command not found


/bin/sh: convert: command not found


### Calculate differential selection for each mutation and each site in NP

Run dms_diffselection on all perturbed proteostasis environments relative to basal conditions at 37 C for each of three replicates.

In [25]:
#REPLICATE 1: -/+ HSF1i 37C
%cd preferences

!dms_diffselection Rep1-DMSO37_counts.txt Rep1-HSF1i37_counts.txt HSF1i37vsDMSO37-rep1_

/Users/annapon/Documents/NP_DMS_2018/2018_NP_DMS/preferences

Beginning execution of dms_diffselection in directory /Users/annapon/Documents/NP_DMS_2018/2018_NP_DMS/preferences

Version information for dms_tools and associated packages.
	Time and date: Sat Mar 24 16:16:27 2018
	Platform: Darwin-15.6.0-x86_64-i386-64bit
	Python version: 2.7.13 |Anaconda 4.4.0 (x86_64)| (default, Dec 20 2016, 23:05:08)  [GCC 4.2.1 Compatible Apple LLVM 6.0 (clang-600.0.57)]
	dms_tools version: 1.2.2
	Bio version: 1.70
	numpy version: 1.12.1
	scipy version: 0.19.0
	matplotlib version: 2.0.2
	cython version: 0.25.2
	pystan version: 2.16.0.0
	weblogolib version: 3.5.0
	PyPDF2 version: 1.26.0
	pandas version: 0.20.1
Parsed the following arguments:
	chartype = codon_to_aa
	pseudocount = 10
	mincounts = 0
	errorcontrolcounts = None
	mockcounts = Rep1-DMSO37_counts.txt
	no_scale_pseudocounts = False
	outprefix = HSF1i37vsDMSO37-rep1_
	selectedcounts = Rep1-HSF1i37_counts.txt
	includestop = False


Read counts d

In [26]:
#REPLICATE 1: -/+ Hsp90i 37C
%cd preferences

!dms_diffselection Rep1-DMSO37_counts.txt Rep1-Hsp90i37_counts.txt Hsp90i37vsDMSO37-rep1_

[Errno 2] No such file or directory: 'preferences'
/Users/annapon/Documents/NP_DMS_2018/2018_NP_DMS/preferences

Beginning execution of dms_diffselection in directory /Users/annapon/Documents/NP_DMS_2018/2018_NP_DMS/preferences

Version information for dms_tools and associated packages.
	Time and date: Sat Mar 24 16:17:23 2018
	Platform: Darwin-15.6.0-x86_64-i386-64bit
	Python version: 2.7.13 |Anaconda 4.4.0 (x86_64)| (default, Dec 20 2016, 23:05:08)  [GCC 4.2.1 Compatible Apple LLVM 6.0 (clang-600.0.57)]
	dms_tools version: 1.2.2
	Bio version: 1.70
	numpy version: 1.12.1
	scipy version: 0.19.0
	matplotlib version: 2.0.2
	cython version: 0.25.2
	pystan version: 2.16.0.0
	weblogolib version: 3.5.0
	PyPDF2 version: 1.26.0
	pandas version: 0.20.1
Parsed the following arguments:
	chartype = codon_to_aa
	pseudocount = 10
	mincounts = 0
	errorcontrolcounts = None
	mockcounts = Rep1-DMSO37_counts.txt
	no_scale_pseudocounts = False
	outprefix = Hsp90i37vsDMSO37-rep1_
	selectedcounts = Rep1-Hsp

In [27]:
#REPLICATE 1: -/+ HSF1i 39C
%cd preferences

!dms_diffselection Rep1-DMSO37_counts.txt Rep1-HSF1i39_counts.txt HSF1i39vsDMSO37-rep1_

[Errno 2] No such file or directory: 'preferences'
/Users/annapon/Documents/NP_DMS_2018/2018_NP_DMS/preferences

Beginning execution of dms_diffselection in directory /Users/annapon/Documents/NP_DMS_2018/2018_NP_DMS/preferences

Version information for dms_tools and associated packages.
	Time and date: Sat Mar 24 16:17:52 2018
	Platform: Darwin-15.6.0-x86_64-i386-64bit
	Python version: 2.7.13 |Anaconda 4.4.0 (x86_64)| (default, Dec 20 2016, 23:05:08)  [GCC 4.2.1 Compatible Apple LLVM 6.0 (clang-600.0.57)]
	dms_tools version: 1.2.2
	Bio version: 1.70
	numpy version: 1.12.1
	scipy version: 0.19.0
	matplotlib version: 2.0.2
	cython version: 0.25.2
	pystan version: 2.16.0.0
	weblogolib version: 3.5.0
	PyPDF2 version: 1.26.0
	pandas version: 0.20.1
Parsed the following arguments:
	chartype = codon_to_aa
	pseudocount = 10
	mincounts = 0
	errorcontrolcounts = None
	mockcounts = Rep1-DMSO37_counts.txt
	no_scale_pseudocounts = False
	outprefix = HSF1i39vsDMSO37-rep1_
	selectedcounts = Rep1-HSF1

In [28]:
#REPLICATE 1: -/+ Hsp90i 39C
%cd preferences

!dms_diffselection Rep1-DMSO37_counts.txt Rep1-Hsp90i39_counts.txt Hsp90i39vsDMSO37-rep1_

[Errno 2] No such file or directory: 'preferences'
/Users/annapon/Documents/NP_DMS_2018/2018_NP_DMS/preferences

Beginning execution of dms_diffselection in directory /Users/annapon/Documents/NP_DMS_2018/2018_NP_DMS/preferences

Version information for dms_tools and associated packages.
	Time and date: Sat Mar 24 16:18:46 2018
	Platform: Darwin-15.6.0-x86_64-i386-64bit
	Python version: 2.7.13 |Anaconda 4.4.0 (x86_64)| (default, Dec 20 2016, 23:05:08)  [GCC 4.2.1 Compatible Apple LLVM 6.0 (clang-600.0.57)]
	dms_tools version: 1.2.2
	Bio version: 1.70
	numpy version: 1.12.1
	scipy version: 0.19.0
	matplotlib version: 2.0.2
	cython version: 0.25.2
	pystan version: 2.16.0.0
	weblogolib version: 3.5.0
	PyPDF2 version: 1.26.0
	pandas version: 0.20.1
Parsed the following arguments:
	chartype = codon_to_aa
	pseudocount = 10
	mincounts = 0
	errorcontrolcounts = None
	mockcounts = Rep1-DMSO37_counts.txt
	no_scale_pseudocounts = False
	outprefix = Hsp90i39vsDMSO37-rep1_
	selectedcounts = Rep1-Hsp

In [29]:
#REPLICATE 1: DMSO 39 vs DMSO 37

!dms_diffselection Rep1-DMSO37_counts.txt Rep1-DMSO39_counts.txt DMSO39vsDMSO37-rep1_


Beginning execution of dms_diffselection in directory /Users/annapon/Documents/NP_DMS_2018/2018_NP_DMS/preferences

Version information for dms_tools and associated packages.
	Time and date: Sat Mar 24 16:20:09 2018
	Platform: Darwin-15.6.0-x86_64-i386-64bit
	Python version: 2.7.13 |Anaconda 4.4.0 (x86_64)| (default, Dec 20 2016, 23:05:08)  [GCC 4.2.1 Compatible Apple LLVM 6.0 (clang-600.0.57)]
	dms_tools version: 1.2.2
	Bio version: 1.70
	numpy version: 1.12.1
	scipy version: 0.19.0
	matplotlib version: 2.0.2
	cython version: 0.25.2
	pystan version: 2.16.0.0
	weblogolib version: 3.5.0
	PyPDF2 version: 1.26.0
	pandas version: 0.20.1
Parsed the following arguments:
	chartype = codon_to_aa
	pseudocount = 10
	mincounts = 0
	errorcontrolcounts = None
	mockcounts = Rep1-DMSO37_counts.txt
	no_scale_pseudocounts = False
	outprefix = DMSO39vsDMSO37-rep1_
	selectedcounts = Rep1-DMSO39_counts.txt
	includestop = False


Read counts data for 499 sites.

Mutations with the lowest differential sele

In [30]:
#REPLICATE 1: HSF1i 39C vs HSF1i 37C

!dms_diffselection Rep1-HSF1i37_counts.txt Rep1-HSF1i39_counts.txt HSF1i39vsHSF1i37-rep1_


Beginning execution of dms_diffselection in directory /Users/annapon/Documents/NP_DMS_2018/2018_NP_DMS/preferences

Version information for dms_tools and associated packages.
	Time and date: Sat Mar 24 16:20:54 2018
	Platform: Darwin-15.6.0-x86_64-i386-64bit
	Python version: 2.7.13 |Anaconda 4.4.0 (x86_64)| (default, Dec 20 2016, 23:05:08)  [GCC 4.2.1 Compatible Apple LLVM 6.0 (clang-600.0.57)]
	dms_tools version: 1.2.2
	Bio version: 1.70
	numpy version: 1.12.1
	scipy version: 0.19.0
	matplotlib version: 2.0.2
	cython version: 0.25.2
	pystan version: 2.16.0.0
	weblogolib version: 3.5.0
	PyPDF2 version: 1.26.0
	pandas version: 0.20.1
Parsed the following arguments:
	chartype = codon_to_aa
	pseudocount = 10
	mincounts = 0
	errorcontrolcounts = None
	mockcounts = Rep1-HSF1i37_counts.txt
	no_scale_pseudocounts = False
	outprefix = HSF1i39vsHSF1i37-rep1_
	selectedcounts = Rep1-HSF1i39_counts.txt
	includestop = False


Read counts data for 499 sites.

Mutations with the lowest differential 

In [31]:
#REPLICATE 1: HSF1i 39C vs DMSO 39C

!dms_diffselection Rep1-DMSO39_counts.txt Rep1-HSF1i39_counts.txt HSF1i39vsDMSO39-rep1_


Beginning execution of dms_diffselection in directory /Users/annapon/Documents/NP_DMS_2018/2018_NP_DMS/preferences

Version information for dms_tools and associated packages.
	Time and date: Sat Mar 24 16:21:47 2018
	Platform: Darwin-15.6.0-x86_64-i386-64bit
	Python version: 2.7.13 |Anaconda 4.4.0 (x86_64)| (default, Dec 20 2016, 23:05:08)  [GCC 4.2.1 Compatible Apple LLVM 6.0 (clang-600.0.57)]
	dms_tools version: 1.2.2
	Bio version: 1.70
	numpy version: 1.12.1
	scipy version: 0.19.0
	matplotlib version: 2.0.2
	cython version: 0.25.2
	pystan version: 2.16.0.0
	weblogolib version: 3.5.0
	PyPDF2 version: 1.26.0
	pandas version: 0.20.1
Parsed the following arguments:
	chartype = codon_to_aa
	pseudocount = 10
	mincounts = 0
	errorcontrolcounts = None
	mockcounts = Rep1-DMSO39_counts.txt
	no_scale_pseudocounts = False
	outprefix = HSF1i39vsDMSO39-rep1_
	selectedcounts = Rep1-HSF1i39_counts.txt
	includestop = False


Read counts data for 499 sites.

Mutations with the lowest differential se

In [32]:
#REPLICATE 2: -/+ HSF1i 37C

!dms_diffselection Rep2-DMSO37_counts.txt Rep2-HSF1i37_counts.txt HSF1i37vsDMSO37-rep2_


Beginning execution of dms_diffselection in directory /Users/annapon/Documents/NP_DMS_2018/2018_NP_DMS/preferences

Version information for dms_tools and associated packages.
	Time and date: Sat Mar 24 16:27:25 2018
	Platform: Darwin-15.6.0-x86_64-i386-64bit
	Python version: 2.7.13 |Anaconda 4.4.0 (x86_64)| (default, Dec 20 2016, 23:05:08)  [GCC 4.2.1 Compatible Apple LLVM 6.0 (clang-600.0.57)]
	dms_tools version: 1.2.2
	Bio version: 1.70
	numpy version: 1.12.1
	scipy version: 0.19.0
	matplotlib version: 2.0.2
	cython version: 0.25.2
	pystan version: 2.16.0.0
	weblogolib version: 3.5.0
	PyPDF2 version: 1.26.0
	pandas version: 0.20.1
Parsed the following arguments:
	chartype = codon_to_aa
	pseudocount = 10
	mincounts = 0
	errorcontrolcounts = None
	mockcounts = Rep2-DMSO37_counts.txt
	no_scale_pseudocounts = False
	outprefix = HSF1i37vsDMSO37-rep2_
	selectedcounts = Rep2-HSF1i37_counts.txt
	includestop = False


Read counts data for 499 sites.

Mutations with the lowest differential se

In [33]:
#REPLICATE 1: -/+ Hsp90i 37C

!dms_diffselection Rep2-DMSO37_counts.txt Rep2-Hsp90i37_counts.txt Hsp90i37vsDMSO37-rep2_


Beginning execution of dms_diffselection in directory /Users/annapon/Documents/NP_DMS_2018/2018_NP_DMS/preferences

Version information for dms_tools and associated packages.
	Time and date: Sat Mar 24 16:28:41 2018
	Platform: Darwin-15.6.0-x86_64-i386-64bit
	Python version: 2.7.13 |Anaconda 4.4.0 (x86_64)| (default, Dec 20 2016, 23:05:08)  [GCC 4.2.1 Compatible Apple LLVM 6.0 (clang-600.0.57)]
	dms_tools version: 1.2.2
	Bio version: 1.70
	numpy version: 1.12.1
	scipy version: 0.19.0
	matplotlib version: 2.0.2
	cython version: 0.25.2
	pystan version: 2.16.0.0
	weblogolib version: 3.5.0
	PyPDF2 version: 1.26.0
	pandas version: 0.20.1
Parsed the following arguments:
	chartype = codon_to_aa
	pseudocount = 10
	mincounts = 0
	errorcontrolcounts = None
	mockcounts = Rep2-DMSO37_counts.txt
	no_scale_pseudocounts = False
	outprefix = Hsp90i37vsDMSO37-rep2_
	selectedcounts = Rep2-Hsp90i37_counts.txt
	includestop = False


Read counts data for 499 sites.

Mutations with the lowest differential 

In [34]:
#REPLICATE 2: -/+ HSF1i 39C

!dms_diffselection Rep2-DMSO37_counts.txt Rep2-HSF1i39_counts.txt HSF1i39vsDMSO37-rep2_


Beginning execution of dms_diffselection in directory /Users/annapon/Documents/NP_DMS_2018/2018_NP_DMS/preferences

Version information for dms_tools and associated packages.
	Time and date: Sat Mar 24 16:29:37 2018
	Platform: Darwin-15.6.0-x86_64-i386-64bit
	Python version: 2.7.13 |Anaconda 4.4.0 (x86_64)| (default, Dec 20 2016, 23:05:08)  [GCC 4.2.1 Compatible Apple LLVM 6.0 (clang-600.0.57)]
	dms_tools version: 1.2.2
	Bio version: 1.70
	numpy version: 1.12.1
	scipy version: 0.19.0
	matplotlib version: 2.0.2
	cython version: 0.25.2
	pystan version: 2.16.0.0
	weblogolib version: 3.5.0
	PyPDF2 version: 1.26.0
	pandas version: 0.20.1
Parsed the following arguments:
	chartype = codon_to_aa
	pseudocount = 10
	mincounts = 0
	errorcontrolcounts = None
	mockcounts = Rep2-DMSO37_counts.txt
	no_scale_pseudocounts = False
	outprefix = HSF1i39vsDMSO37-rep2_
	selectedcounts = Rep2-HSF1i39_counts.txt
	includestop = False


Read counts data for 499 sites.

Mutations with the lowest differential se

In [36]:
#REPLICATE 2: -/+ Hsp90i 39C

!dms_diffselection Rep2-DMSO37_counts.txt Rep2-Hsp90i39_counts.txt Hsp90i39vsDMSO37-rep2_


Beginning execution of dms_diffselection in directory /Users/annapon/Documents/NP_DMS_2018/2018_NP_DMS/preferences

Version information for dms_tools and associated packages.
	Time and date: Sat Mar 24 16:30:41 2018
	Platform: Darwin-15.6.0-x86_64-i386-64bit
	Python version: 2.7.13 |Anaconda 4.4.0 (x86_64)| (default, Dec 20 2016, 23:05:08)  [GCC 4.2.1 Compatible Apple LLVM 6.0 (clang-600.0.57)]
	dms_tools version: 1.2.2
	Bio version: 1.70
	numpy version: 1.12.1
	scipy version: 0.19.0
	matplotlib version: 2.0.2
	cython version: 0.25.2
	pystan version: 2.16.0.0
	weblogolib version: 3.5.0
	PyPDF2 version: 1.26.0
	pandas version: 0.20.1
Parsed the following arguments:
	chartype = codon_to_aa
	pseudocount = 10
	mincounts = 0
	errorcontrolcounts = None
	mockcounts = Rep2-DMSO37_counts.txt
	no_scale_pseudocounts = False
	outprefix = Hsp90i39vsDMSO37-rep2_
	selectedcounts = Rep2-Hsp90i39_counts.txt
	includestop = False


Read counts data for 499 sites.

Mutations with the lowest differential 

In [37]:
#REPLICATE 2: DMSO 39C vs DMSO 37C

!dms_diffselection Rep2-DMSO37_counts.txt Rep2-DMSO39_counts.txt DMSO39vsDMSO37-rep2_


Beginning execution of dms_diffselection in directory /Users/annapon/Documents/NP_DMS_2018/2018_NP_DMS/preferences

Version information for dms_tools and associated packages.
	Time and date: Sat Mar 24 16:31:46 2018
	Platform: Darwin-15.6.0-x86_64-i386-64bit
	Python version: 2.7.13 |Anaconda 4.4.0 (x86_64)| (default, Dec 20 2016, 23:05:08)  [GCC 4.2.1 Compatible Apple LLVM 6.0 (clang-600.0.57)]
	dms_tools version: 1.2.2
	Bio version: 1.70
	numpy version: 1.12.1
	scipy version: 0.19.0
	matplotlib version: 2.0.2
	cython version: 0.25.2
	pystan version: 2.16.0.0
	weblogolib version: 3.5.0
	PyPDF2 version: 1.26.0
	pandas version: 0.20.1
Parsed the following arguments:
	chartype = codon_to_aa
	pseudocount = 10
	mincounts = 0
	errorcontrolcounts = None
	mockcounts = Rep2-DMSO37_counts.txt
	no_scale_pseudocounts = False
	outprefix = DMSO39vsDMSO37-rep2_
	selectedcounts = Rep2-DMSO39_counts.txt
	includestop = False


Read counts data for 499 sites.

Mutations with the lowest differential sele

In [38]:
#REPLICATE 2: HSF1i 39C vs HSF1i 37C

!dms_diffselection Rep2-HSF1i37_counts.txt Rep2-HSF1i39_counts.txt HSF1i39vsHSF1i37-rep2_


Beginning execution of dms_diffselection in directory /Users/annapon/Documents/NP_DMS_2018/2018_NP_DMS/preferences

Version information for dms_tools and associated packages.
	Time and date: Sat Mar 24 16:32:43 2018
	Platform: Darwin-15.6.0-x86_64-i386-64bit
	Python version: 2.7.13 |Anaconda 4.4.0 (x86_64)| (default, Dec 20 2016, 23:05:08)  [GCC 4.2.1 Compatible Apple LLVM 6.0 (clang-600.0.57)]
	dms_tools version: 1.2.2
	Bio version: 1.70
	numpy version: 1.12.1
	scipy version: 0.19.0
	matplotlib version: 2.0.2
	cython version: 0.25.2
	pystan version: 2.16.0.0
	weblogolib version: 3.5.0
	PyPDF2 version: 1.26.0
	pandas version: 0.20.1
Parsed the following arguments:
	chartype = codon_to_aa
	pseudocount = 10
	mincounts = 0
	errorcontrolcounts = None
	mockcounts = Rep2-HSF1i37_counts.txt
	no_scale_pseudocounts = False
	outprefix = HSF1i39vsHSF1i37-rep2_
	selectedcounts = Rep2-HSF1i39_counts.txt
	includestop = False


Read counts data for 499 sites.

Mutations with the lowest differential 

In [39]:
#REPLICATE 2: HSF1i 39C vs DMSO 39C

!dms_diffselection Rep2-DMSO39_counts.txt Rep2-HSF1i39_counts.txt HSF1i39vsDMSO39-rep2_


Beginning execution of dms_diffselection in directory /Users/annapon/Documents/NP_DMS_2018/2018_NP_DMS/preferences

Version information for dms_tools and associated packages.
	Time and date: Sat Mar 24 16:33:39 2018
	Platform: Darwin-15.6.0-x86_64-i386-64bit
	Python version: 2.7.13 |Anaconda 4.4.0 (x86_64)| (default, Dec 20 2016, 23:05:08)  [GCC 4.2.1 Compatible Apple LLVM 6.0 (clang-600.0.57)]
	dms_tools version: 1.2.2
	Bio version: 1.70
	numpy version: 1.12.1
	scipy version: 0.19.0
	matplotlib version: 2.0.2
	cython version: 0.25.2
	pystan version: 2.16.0.0
	weblogolib version: 3.5.0
	PyPDF2 version: 1.26.0
	pandas version: 0.20.1
Parsed the following arguments:
	chartype = codon_to_aa
	pseudocount = 10
	mincounts = 0
	errorcontrolcounts = None
	mockcounts = Rep2-DMSO39_counts.txt
	no_scale_pseudocounts = False
	outprefix = HSF1i39vsDMSO39-rep2_
	selectedcounts = Rep2-HSF1i39_counts.txt
	includestop = False


Read counts data for 499 sites.

Mutations with the lowest differential se

In [40]:
#REPLICATE 3: -/+ HSF1i 37C

!dms_diffselection Rep3-DMSO37_counts.txt Rep3-HSF1i37_counts.txt HSF1i37vsDMSO37-rep3_


Beginning execution of dms_diffselection in directory /Users/annapon/Documents/NP_DMS_2018/2018_NP_DMS/preferences

Version information for dms_tools and associated packages.
	Time and date: Sat Mar 24 16:34:15 2018
	Platform: Darwin-15.6.0-x86_64-i386-64bit
	Python version: 2.7.13 |Anaconda 4.4.0 (x86_64)| (default, Dec 20 2016, 23:05:08)  [GCC 4.2.1 Compatible Apple LLVM 6.0 (clang-600.0.57)]
	dms_tools version: 1.2.2
	Bio version: 1.70
	numpy version: 1.12.1
	scipy version: 0.19.0
	matplotlib version: 2.0.2
	cython version: 0.25.2
	pystan version: 2.16.0.0
	weblogolib version: 3.5.0
	PyPDF2 version: 1.26.0
	pandas version: 0.20.1
Parsed the following arguments:
	chartype = codon_to_aa
	pseudocount = 10
	mincounts = 0
	errorcontrolcounts = None
	mockcounts = Rep3-DMSO37_counts.txt
	no_scale_pseudocounts = False
	outprefix = HSF1i37vsDMSO37-rep3_
	selectedcounts = Rep3-HSF1i37_counts.txt
	includestop = False


Read counts data for 499 sites.

Mutations with the lowest differential se

In [41]:
#REPLICATE 3: -/+ Hsp90i 37C

!dms_diffselection Rep3-DMSO37_counts.txt Rep3-HSF1i37_counts.txt HSF1i37vsDMSO37-rep3_


Beginning execution of dms_diffselection in directory /Users/annapon/Documents/NP_DMS_2018/2018_NP_DMS/preferences

Version information for dms_tools and associated packages.
	Time and date: Sat Mar 24 16:34:39 2018
	Platform: Darwin-15.6.0-x86_64-i386-64bit
	Python version: 2.7.13 |Anaconda 4.4.0 (x86_64)| (default, Dec 20 2016, 23:05:08)  [GCC 4.2.1 Compatible Apple LLVM 6.0 (clang-600.0.57)]
	dms_tools version: 1.2.2
	Bio version: 1.70
	numpy version: 1.12.1
	scipy version: 0.19.0
	matplotlib version: 2.0.2
	cython version: 0.25.2
	pystan version: 2.16.0.0
	weblogolib version: 3.5.0
	PyPDF2 version: 1.26.0
	pandas version: 0.20.1
Parsed the following arguments:
	chartype = codon_to_aa
	pseudocount = 10
	mincounts = 0
	errorcontrolcounts = None
	mockcounts = Rep3-DMSO37_counts.txt
	no_scale_pseudocounts = False
	outprefix = HSF1i37vsDMSO37-rep3_
	selectedcounts = Rep3-HSF1i37_counts.txt
	includestop = False


Removing existing output file HSF1i37vsDMSO37-rep3_mutdiffsel.txt

Removin

In [42]:
#REPLICATE 3: -/+ HSF1i 39C

!dms_diffselection Rep3-DMSO37_counts.txt Rep3-HSF1i39_counts.txt HSF1i39vsDMSO37-rep3_


Beginning execution of dms_diffselection in directory /Users/annapon/Documents/NP_DMS_2018/2018_NP_DMS/preferences

Version information for dms_tools and associated packages.
	Time and date: Sat Mar 24 16:35:08 2018
	Platform: Darwin-15.6.0-x86_64-i386-64bit
	Python version: 2.7.13 |Anaconda 4.4.0 (x86_64)| (default, Dec 20 2016, 23:05:08)  [GCC 4.2.1 Compatible Apple LLVM 6.0 (clang-600.0.57)]
	dms_tools version: 1.2.2
	Bio version: 1.70
	numpy version: 1.12.1
	scipy version: 0.19.0
	matplotlib version: 2.0.2
	cython version: 0.25.2
	pystan version: 2.16.0.0
	weblogolib version: 3.5.0
	PyPDF2 version: 1.26.0
	pandas version: 0.20.1
Parsed the following arguments:
	chartype = codon_to_aa
	pseudocount = 10
	mincounts = 0
	errorcontrolcounts = None
	mockcounts = Rep3-DMSO37_counts.txt
	no_scale_pseudocounts = False
	outprefix = HSF1i39vsDMSO37-rep3_
	selectedcounts = Rep3-HSF1i39_counts.txt
	includestop = False


Read counts data for 499 sites.

Mutations with the lowest differential se

In [43]:
#REPLICATE 3: -/+ Hsp90i 39C

!dms_diffselection Rep3-DMSO37_counts.txt Rep3-Hsp90i39_counts.txt Hsp90i39vsDMSO37-rep3_


Beginning execution of dms_diffselection in directory /Users/annapon/Documents/NP_DMS_2018/2018_NP_DMS/preferences

Version information for dms_tools and associated packages.
	Time and date: Sat Mar 24 16:36:00 2018
	Platform: Darwin-15.6.0-x86_64-i386-64bit
	Python version: 2.7.13 |Anaconda 4.4.0 (x86_64)| (default, Dec 20 2016, 23:05:08)  [GCC 4.2.1 Compatible Apple LLVM 6.0 (clang-600.0.57)]
	dms_tools version: 1.2.2
	Bio version: 1.70
	numpy version: 1.12.1
	scipy version: 0.19.0
	matplotlib version: 2.0.2
	cython version: 0.25.2
	pystan version: 2.16.0.0
	weblogolib version: 3.5.0
	PyPDF2 version: 1.26.0
	pandas version: 0.20.1
Parsed the following arguments:
	chartype = codon_to_aa
	pseudocount = 10
	mincounts = 0
	errorcontrolcounts = None
	mockcounts = Rep3-DMSO37_counts.txt
	no_scale_pseudocounts = False
	outprefix = Hsp90i39vsDMSO37-rep3_
	selectedcounts = Rep3-Hsp90i39_counts.txt
	includestop = False


Read counts data for 499 sites.

Mutations with the lowest differential 

In [44]:
#REPLICATE 3: DMSO 39C vs DMSO 37C

!dms_diffselection Rep3-DMSO37_counts.txt Rep3-DMSO39_counts.txt DMSO39vsDMSO37-rep3_


Beginning execution of dms_diffselection in directory /Users/annapon/Documents/NP_DMS_2018/2018_NP_DMS/preferences

Version information for dms_tools and associated packages.
	Time and date: Sat Mar 24 16:36:52 2018
	Platform: Darwin-15.6.0-x86_64-i386-64bit
	Python version: 2.7.13 |Anaconda 4.4.0 (x86_64)| (default, Dec 20 2016, 23:05:08)  [GCC 4.2.1 Compatible Apple LLVM 6.0 (clang-600.0.57)]
	dms_tools version: 1.2.2
	Bio version: 1.70
	numpy version: 1.12.1
	scipy version: 0.19.0
	matplotlib version: 2.0.2
	cython version: 0.25.2
	pystan version: 2.16.0.0
	weblogolib version: 3.5.0
	PyPDF2 version: 1.26.0
	pandas version: 0.20.1
Parsed the following arguments:
	chartype = codon_to_aa
	pseudocount = 10
	mincounts = 0
	errorcontrolcounts = None
	mockcounts = Rep3-DMSO37_counts.txt
	no_scale_pseudocounts = False
	outprefix = DMSO39vsDMSO37-rep3_
	selectedcounts = Rep3-DMSO39_counts.txt
	includestop = False


Read counts data for 499 sites.

Mutations with the lowest differential sele

In [45]:
#REPLICATE 3: HSF1i 39C vs HSF1i 37C

!dms_diffselection Rep3-HSF1i37_counts.txt Rep3-HSF1i39_counts.txt HSF1i39vsDHSF1i37-rep3_


Beginning execution of dms_diffselection in directory /Users/annapon/Documents/NP_DMS_2018/2018_NP_DMS/preferences

Version information for dms_tools and associated packages.
	Time and date: Sat Mar 24 16:37:45 2018
	Platform: Darwin-15.6.0-x86_64-i386-64bit
	Python version: 2.7.13 |Anaconda 4.4.0 (x86_64)| (default, Dec 20 2016, 23:05:08)  [GCC 4.2.1 Compatible Apple LLVM 6.0 (clang-600.0.57)]
	dms_tools version: 1.2.2
	Bio version: 1.70
	numpy version: 1.12.1
	scipy version: 0.19.0
	matplotlib version: 2.0.2
	cython version: 0.25.2
	pystan version: 2.16.0.0
	weblogolib version: 3.5.0
	PyPDF2 version: 1.26.0
	pandas version: 0.20.1
Parsed the following arguments:
	chartype = codon_to_aa
	pseudocount = 10
	mincounts = 0
	errorcontrolcounts = None
	mockcounts = Rep3-HSF1i37_counts.txt
	no_scale_pseudocounts = False
	outprefix = HSF1i39vsDHSF1i37-rep3_
	selectedcounts = Rep3-HSF1i39_counts.txt
	includestop = False


Read counts data for 499 sites.

Mutations with the lowest differential

In [46]:
#REPLICATE 3: HSF1i 39C vs DMSO 39C

!dms_diffselection Rep3-DMSO39_counts.txt Rep3-HSF1i39_counts.txt HSF1i39vsDMSO39-rep3_


Beginning execution of dms_diffselection in directory /Users/annapon/Documents/NP_DMS_2018/2018_NP_DMS/preferences

Version information for dms_tools and associated packages.
	Time and date: Sat Mar 24 16:38:29 2018
	Platform: Darwin-15.6.0-x86_64-i386-64bit
	Python version: 2.7.13 |Anaconda 4.4.0 (x86_64)| (default, Dec 20 2016, 23:05:08)  [GCC 4.2.1 Compatible Apple LLVM 6.0 (clang-600.0.57)]
	dms_tools version: 1.2.2
	Bio version: 1.70
	numpy version: 1.12.1
	scipy version: 0.19.0
	matplotlib version: 2.0.2
	cython version: 0.25.2
	pystan version: 2.16.0.0
	weblogolib version: 3.5.0
	PyPDF2 version: 1.26.0
	pandas version: 0.20.1
Parsed the following arguments:
	chartype = codon_to_aa
	pseudocount = 10
	mincounts = 0
	errorcontrolcounts = None
	mockcounts = Rep3-DMSO39_counts.txt
	no_scale_pseudocounts = False
	outprefix = HSF1i39vsDMSO39-rep3_
	selectedcounts = Rep3-HSF1i39_counts.txt
	includestop = False


Read counts data for 499 sites.

Mutations with the lowest differential se

### Generate logo plots

In [31]:
#make logoplots for each selection, same axes for all

#rep1 HSF1i 39C vs DMSO 37C
!dms_logoplot HSF1i39vsDMSO37-rep1_mutdiffsel.txt HSF1i39vsDMSO37-rep1_diffsel_logoplot.pdf --nperline 100 --mapmetric charge --letterheight 2 --diffselheight DMSO39vsDMSO37-rep1_mutdiffsel.txt DMSO39vsDMSO37-rep2_mutdiffsel.txt DMSO39vsDMSO37-rep3_mutdiffsel.txt HSF1i37vsDMSO37-rep1_mutdiffsel.txt HSF1i37vsDMSO37-rep2_mutdiffsel.txt HSF1i37vsDMSO37-rep3_mutdiffsel.txt HSF1i39vsDMSO37-rep1_mutdiffsel.txt HSF1i39vsDMSO37-rep2_mutdiffsel.txt HSF1i39vsDMSO37-rep3_mutdiffsel.txt HSF1i39vsDMSO39-rep1_mutdiffsel.txt HSF1i39vsDMSO39-rep2_mutdiffsel.txt HSF1i39vsDMSO39-rep3_mutdiffsel.txt HSF1i39vsHSF1i37-rep1_mutdiffsel.txt HSF1i39vsHSF1i37-rep2_mutdiffsel.txt HSF1i39vsHSF1i37-rep3_mutdiffsel.txt Hsp90i37vsDMSO37-rep1_mutdiffsel.txt Hsp90i37vsDMSO37-rep2_mutdiffsel.txt Hsp90i39vsDMSO37-rep3_mutdiffsel.txt Hsp90i39vsDMSO37-rep1_mutdiffsel.txt Hsp90i39vsDMSO37-rep2_mutdiffsel.txt Hsp90i39vsDMSO37-rep3_mutdiffsel.txt  


Beginning execution of dms_logoplot in directory /Users/annapon/Documents/NP_DMS_2018/2018_NP_DMS/preferences

Version information for dms_tools and associated packages.
	Time and date: Tue Mar 27 18:55:35 2018
	Platform: Darwin-15.6.0-x86_64-i386-64bit
	Python version: 2.7.13 |Anaconda 4.4.0 (x86_64)| (default, Dec 20 2016, 23:05:08)  [GCC 4.2.1 Compatible Apple LLVM 6.0 (clang-600.0.57)]
	dms_tools version: 1.2.2
	Bio version: 1.70
	numpy version: 1.12.1
	scipy version: 0.19.0
	matplotlib version: 2.0.2
	cython version: 0.25.2
	pystan version: 2.16.0.0
	weblogolib version: 3.5.0
	PyPDF2 version: 1.26.0
	pandas version: 0.20.1

Parsed the following arguments:
	restrictdiffsel = None
	overlay3 = None
	colormap = jet
	nperline = 100
	numberevery = 10
	excludestop = False
	logoplot = HSF1i39vsDMSO37-rep1_diffsel_logoplot.pdf
	overlay_cmap = None
	mapmetric = charge
	diffprefheight = 1.0
	nosepline = False
	overlay1 = None
	overlay2 = None
	diffselheight = ['DMSO39vsDMSO37-rep1_mutdiffse

In [32]:
#rep2 HSF1i 39C vs DMSO 37C
!dms_logoplot HSF1i39vsDMSO37-rep2_mutdiffsel.txt HSF1i39vsDMSO37-rep2_diffsel_logoplot.pdf --nperline 100 --mapmetric charge --letterheight 2 --diffselheight DMSO39vsDMSO37-rep1_mutdiffsel.txt DMSO39vsDMSO37-rep2_mutdiffsel.txt DMSO39vsDMSO37-rep3_mutdiffsel.txt HSF1i37vsDMSO37-rep1_mutdiffsel.txt HSF1i37vsDMSO37-rep2_mutdiffsel.txt HSF1i37vsDMSO37-rep3_mutdiffsel.txt HSF1i39vsDMSO37-rep1_mutdiffsel.txt HSF1i39vsDMSO37-rep2_mutdiffsel.txt HSF1i39vsDMSO37-rep3_mutdiffsel.txt HSF1i39vsDMSO39-rep1_mutdiffsel.txt HSF1i39vsDMSO39-rep2_mutdiffsel.txt HSF1i39vsDMSO39-rep3_mutdiffsel.txt HSF1i39vsHSF1i37-rep1_mutdiffsel.txt HSF1i39vsHSF1i37-rep2_mutdiffsel.txt HSF1i39vsHSF1i37-rep3_mutdiffsel.txt Hsp90i37vsDMSO37-rep1_mutdiffsel.txt Hsp90i37vsDMSO37-rep2_mutdiffsel.txt Hsp90i39vsDMSO37-rep3_mutdiffsel.txt Hsp90i39vsDMSO37-rep1_mutdiffsel.txt Hsp90i39vsDMSO37-rep2_mutdiffsel.txt Hsp90i39vsDMSO37-rep3_mutdiffsel.txt  


Beginning execution of dms_logoplot in directory /Users/annapon/Documents/NP_DMS_2018/2018_NP_DMS/preferences

Version information for dms_tools and associated packages.
	Time and date: Tue Mar 27 18:56:18 2018
	Platform: Darwin-15.6.0-x86_64-i386-64bit
	Python version: 2.7.13 |Anaconda 4.4.0 (x86_64)| (default, Dec 20 2016, 23:05:08)  [GCC 4.2.1 Compatible Apple LLVM 6.0 (clang-600.0.57)]
	dms_tools version: 1.2.2
	Bio version: 1.70
	numpy version: 1.12.1
	scipy version: 0.19.0
	matplotlib version: 2.0.2
	cython version: 0.25.2
	pystan version: 2.16.0.0
	weblogolib version: 3.5.0
	PyPDF2 version: 1.26.0
	pandas version: 0.20.1

Parsed the following arguments:
	restrictdiffsel = None
	overlay3 = None
	colormap = jet
	nperline = 100
	numberevery = 10
	excludestop = False
	logoplot = HSF1i39vsDMSO37-rep2_diffsel_logoplot.pdf
	overlay_cmap = None
	mapmetric = charge
	diffprefheight = 1.0
	nosepline = False
	overlay1 = None
	overlay2 = None
	diffselheight = ['DMSO39vsDMSO37-rep1_mutdiffse

In [33]:
#rep3 HSF1i 39C vs DMSO 37C
!dms_logoplot HSF1i39vsDMSO37-rep3_mutdiffsel.txt HSF1i39vsDMSO37-rep3_diffsel_logoplot.pdf --nperline 100 --mapmetric charge --letterheight 2 --diffselheight DMSO39vsDMSO37-rep1_mutdiffsel.txt DMSO39vsDMSO37-rep2_mutdiffsel.txt DMSO39vsDMSO37-rep3_mutdiffsel.txt HSF1i37vsDMSO37-rep1_mutdiffsel.txt HSF1i37vsDMSO37-rep2_mutdiffsel.txt HSF1i37vsDMSO37-rep3_mutdiffsel.txt HSF1i39vsDMSO37-rep1_mutdiffsel.txt HSF1i39vsDMSO37-rep2_mutdiffsel.txt HSF1i39vsDMSO37-rep3_mutdiffsel.txt HSF1i39vsDMSO39-rep1_mutdiffsel.txt HSF1i39vsDMSO39-rep2_mutdiffsel.txt HSF1i39vsDMSO39-rep3_mutdiffsel.txt HSF1i39vsHSF1i37-rep1_mutdiffsel.txt HSF1i39vsHSF1i37-rep2_mutdiffsel.txt HSF1i39vsHSF1i37-rep3_mutdiffsel.txt Hsp90i37vsDMSO37-rep1_mutdiffsel.txt Hsp90i37vsDMSO37-rep2_mutdiffsel.txt Hsp90i39vsDMSO37-rep3_mutdiffsel.txt Hsp90i39vsDMSO37-rep1_mutdiffsel.txt Hsp90i39vsDMSO37-rep2_mutdiffsel.txt Hsp90i39vsDMSO37-rep3_mutdiffsel.txt  


Beginning execution of dms_logoplot in directory /Users/annapon/Documents/NP_DMS_2018/2018_NP_DMS/preferences

Version information for dms_tools and associated packages.
	Time and date: Tue Mar 27 18:56:37 2018
	Platform: Darwin-15.6.0-x86_64-i386-64bit
	Python version: 2.7.13 |Anaconda 4.4.0 (x86_64)| (default, Dec 20 2016, 23:05:08)  [GCC 4.2.1 Compatible Apple LLVM 6.0 (clang-600.0.57)]
	dms_tools version: 1.2.2
	Bio version: 1.70
	numpy version: 1.12.1
	scipy version: 0.19.0
	matplotlib version: 2.0.2
	cython version: 0.25.2
	pystan version: 2.16.0.0
	weblogolib version: 3.5.0
	PyPDF2 version: 1.26.0
	pandas version: 0.20.1

Parsed the following arguments:
	restrictdiffsel = None
	overlay3 = None
	colormap = jet
	nperline = 100
	numberevery = 10
	excludestop = False
	logoplot = HSF1i39vsDMSO37-rep3_diffsel_logoplot.pdf
	overlay_cmap = None
	mapmetric = charge
	diffprefheight = 1.0
	nosepline = False
	overlay1 = None
	overlay2 = None
	diffselheight = ['DMSO39vsDMSO37-rep1_mutdiffse

In [34]:
#rep1 DMSO 39C vs DMSO 37C
!dms_logoplot DMSO39vsDMSO37-rep1_mutdiffsel.txt DMSO39vsDMSO37-rep1_diffsel_logoplot.pdf --nperline 100 --mapmetric charge --letterheight 2 --diffselheight DMSO39vsDMSO37-rep1_mutdiffsel.txt DMSO39vsDMSO37-rep2_mutdiffsel.txt DMSO39vsDMSO37-rep3_mutdiffsel.txt HSF1i37vsDMSO37-rep1_mutdiffsel.txt HSF1i37vsDMSO37-rep2_mutdiffsel.txt HSF1i37vsDMSO37-rep3_mutdiffsel.txt HSF1i39vsDMSO37-rep1_mutdiffsel.txt HSF1i39vsDMSO37-rep2_mutdiffsel.txt HSF1i39vsDMSO37-rep3_mutdiffsel.txt HSF1i39vsDMSO39-rep1_mutdiffsel.txt HSF1i39vsDMSO39-rep2_mutdiffsel.txt HSF1i39vsDMSO39-rep3_mutdiffsel.txt HSF1i39vsHSF1i37-rep1_mutdiffsel.txt HSF1i39vsHSF1i37-rep2_mutdiffsel.txt HSF1i39vsHSF1i37-rep3_mutdiffsel.txt Hsp90i37vsDMSO37-rep1_mutdiffsel.txt Hsp90i37vsDMSO37-rep2_mutdiffsel.txt Hsp90i39vsDMSO37-rep3_mutdiffsel.txt Hsp90i39vsDMSO37-rep1_mutdiffsel.txt Hsp90i39vsDMSO37-rep2_mutdiffsel.txt Hsp90i39vsDMSO37-rep3_mutdiffsel.txt  


Beginning execution of dms_logoplot in directory /Users/annapon/Documents/NP_DMS_2018/2018_NP_DMS/preferences

Version information for dms_tools and associated packages.
	Time and date: Tue Mar 27 18:56:55 2018
	Platform: Darwin-15.6.0-x86_64-i386-64bit
	Python version: 2.7.13 |Anaconda 4.4.0 (x86_64)| (default, Dec 20 2016, 23:05:08)  [GCC 4.2.1 Compatible Apple LLVM 6.0 (clang-600.0.57)]
	dms_tools version: 1.2.2
	Bio version: 1.70
	numpy version: 1.12.1
	scipy version: 0.19.0
	matplotlib version: 2.0.2
	cython version: 0.25.2
	pystan version: 2.16.0.0
	weblogolib version: 3.5.0
	PyPDF2 version: 1.26.0
	pandas version: 0.20.1

Parsed the following arguments:
	restrictdiffsel = None
	overlay3 = None
	colormap = jet
	nperline = 100
	numberevery = 10
	excludestop = False
	logoplot = DMSO39vsDMSO37-rep1_diffsel_logoplot.pdf
	overlay_cmap = None
	mapmetric = charge
	diffprefheight = 1.0
	nosepline = False
	overlay1 = None
	overlay2 = None
	diffselheight = ['DMSO39vsDMSO37-rep1_mutdiffsel

In [35]:
#rep2 DMSO 39C vs DMSO 37C
!dms_logoplot DMSO39vsDMSO37-rep2_mutdiffsel.txt DMSO39vsDMSO37-rep2_diffsel_logoplot.pdf --nperline 100 --mapmetric charge --letterheight 2 --diffselheight DMSO39vsDMSO37-rep1_mutdiffsel.txt DMSO39vsDMSO37-rep2_mutdiffsel.txt DMSO39vsDMSO37-rep3_mutdiffsel.txt HSF1i37vsDMSO37-rep1_mutdiffsel.txt HSF1i37vsDMSO37-rep2_mutdiffsel.txt HSF1i37vsDMSO37-rep3_mutdiffsel.txt HSF1i39vsDMSO37-rep1_mutdiffsel.txt HSF1i39vsDMSO37-rep2_mutdiffsel.txt HSF1i39vsDMSO37-rep3_mutdiffsel.txt HSF1i39vsDMSO39-rep1_mutdiffsel.txt HSF1i39vsDMSO39-rep2_mutdiffsel.txt HSF1i39vsDMSO39-rep3_mutdiffsel.txt HSF1i39vsHSF1i37-rep1_mutdiffsel.txt HSF1i39vsHSF1i37-rep2_mutdiffsel.txt HSF1i39vsHSF1i37-rep3_mutdiffsel.txt Hsp90i37vsDMSO37-rep1_mutdiffsel.txt Hsp90i37vsDMSO37-rep2_mutdiffsel.txt Hsp90i39vsDMSO37-rep3_mutdiffsel.txt Hsp90i39vsDMSO37-rep1_mutdiffsel.txt Hsp90i39vsDMSO37-rep2_mutdiffsel.txt Hsp90i39vsDMSO37-rep3_mutdiffsel.txt  


Beginning execution of dms_logoplot in directory /Users/annapon/Documents/NP_DMS_2018/2018_NP_DMS/preferences

Version information for dms_tools and associated packages.
	Time and date: Tue Mar 27 18:57:40 2018
	Platform: Darwin-15.6.0-x86_64-i386-64bit
	Python version: 2.7.13 |Anaconda 4.4.0 (x86_64)| (default, Dec 20 2016, 23:05:08)  [GCC 4.2.1 Compatible Apple LLVM 6.0 (clang-600.0.57)]
	dms_tools version: 1.2.2
	Bio version: 1.70
	numpy version: 1.12.1
	scipy version: 0.19.0
	matplotlib version: 2.0.2
	cython version: 0.25.2
	pystan version: 2.16.0.0
	weblogolib version: 3.5.0
	PyPDF2 version: 1.26.0
	pandas version: 0.20.1

Parsed the following arguments:
	restrictdiffsel = None
	overlay3 = None
	colormap = jet
	nperline = 100
	numberevery = 10
	excludestop = False
	logoplot = DMSO39vsDMSO37-rep2_diffsel_logoplot.pdf
	overlay_cmap = None
	mapmetric = charge
	diffprefheight = 1.0
	nosepline = False
	overlay1 = None
	overlay2 = None
	diffselheight = ['DMSO39vsDMSO37-rep1_mutdiffsel

In [36]:
#rep3 DMSO 39C vs DMSO 37C
!dms_logoplot DMSO39vsDMSO37-rep3_mutdiffsel.txt DMSO39vsDMSO37-rep3_diffsel_logoplot.pdf --nperline 100 --mapmetric charge --letterheight 2 --diffselheight DMSO39vsDMSO37-rep1_mutdiffsel.txt DMSO39vsDMSO37-rep2_mutdiffsel.txt DMSO39vsDMSO37-rep3_mutdiffsel.txt HSF1i37vsDMSO37-rep1_mutdiffsel.txt HSF1i37vsDMSO37-rep2_mutdiffsel.txt HSF1i37vsDMSO37-rep3_mutdiffsel.txt HSF1i39vsDMSO37-rep1_mutdiffsel.txt HSF1i39vsDMSO37-rep2_mutdiffsel.txt HSF1i39vsDMSO37-rep3_mutdiffsel.txt HSF1i39vsDMSO39-rep1_mutdiffsel.txt HSF1i39vsDMSO39-rep2_mutdiffsel.txt HSF1i39vsDMSO39-rep3_mutdiffsel.txt HSF1i39vsHSF1i37-rep1_mutdiffsel.txt HSF1i39vsHSF1i37-rep2_mutdiffsel.txt HSF1i39vsHSF1i37-rep3_mutdiffsel.txt Hsp90i37vsDMSO37-rep1_mutdiffsel.txt Hsp90i37vsDMSO37-rep2_mutdiffsel.txt Hsp90i39vsDMSO37-rep3_mutdiffsel.txt Hsp90i39vsDMSO37-rep1_mutdiffsel.txt Hsp90i39vsDMSO37-rep2_mutdiffsel.txt Hsp90i39vsDMSO37-rep3_mutdiffsel.txt  


Beginning execution of dms_logoplot in directory /Users/annapon/Documents/NP_DMS_2018/2018_NP_DMS/preferences

Version information for dms_tools and associated packages.
	Time and date: Tue Mar 27 18:57:59 2018
	Platform: Darwin-15.6.0-x86_64-i386-64bit
	Python version: 2.7.13 |Anaconda 4.4.0 (x86_64)| (default, Dec 20 2016, 23:05:08)  [GCC 4.2.1 Compatible Apple LLVM 6.0 (clang-600.0.57)]
	dms_tools version: 1.2.2
	Bio version: 1.70
	numpy version: 1.12.1
	scipy version: 0.19.0
	matplotlib version: 2.0.2
	cython version: 0.25.2
	pystan version: 2.16.0.0
	weblogolib version: 3.5.0
	PyPDF2 version: 1.26.0
	pandas version: 0.20.1

Parsed the following arguments:
	restrictdiffsel = None
	overlay3 = None
	colormap = jet
	nperline = 100
	numberevery = 10
	excludestop = False
	logoplot = DMSO39vsDMSO37-rep3_diffsel_logoplot.pdf
	overlay_cmap = None
	mapmetric = charge
	diffprefheight = 1.0
	nosepline = False
	overlay1 = None
	overlay2 = None
	diffselheight = ['DMSO39vsDMSO37-rep1_mutdiffsel

In [37]:
#rep1 HSF1i 37C vs DMSO 37C
!dms_logoplot HSF1i37vsDMSO37-rep1_mutdiffsel.txt HSF1i37vsDMSO37-rep1_diffsel_logoplot.pdf --nperline 100 --mapmetric charge --letterheight 2 --diffselheight DMSO39vsDMSO37-rep1_mutdiffsel.txt DMSO39vsDMSO37-rep2_mutdiffsel.txt DMSO39vsDMSO37-rep3_mutdiffsel.txt HSF1i37vsDMSO37-rep1_mutdiffsel.txt HSF1i37vsDMSO37-rep2_mutdiffsel.txt HSF1i37vsDMSO37-rep3_mutdiffsel.txt HSF1i39vsDMSO37-rep1_mutdiffsel.txt HSF1i39vsDMSO37-rep2_mutdiffsel.txt HSF1i39vsDMSO37-rep3_mutdiffsel.txt HSF1i39vsDMSO39-rep1_mutdiffsel.txt HSF1i39vsDMSO39-rep2_mutdiffsel.txt HSF1i39vsDMSO39-rep3_mutdiffsel.txt HSF1i39vsHSF1i37-rep1_mutdiffsel.txt HSF1i39vsHSF1i37-rep2_mutdiffsel.txt HSF1i39vsHSF1i37-rep3_mutdiffsel.txt Hsp90i37vsDMSO37-rep1_mutdiffsel.txt Hsp90i37vsDMSO37-rep2_mutdiffsel.txt Hsp90i39vsDMSO37-rep3_mutdiffsel.txt Hsp90i39vsDMSO37-rep1_mutdiffsel.txt Hsp90i39vsDMSO37-rep2_mutdiffsel.txt Hsp90i39vsDMSO37-rep3_mutdiffsel.txt  


Beginning execution of dms_logoplot in directory /Users/annapon/Documents/NP_DMS_2018/2018_NP_DMS/preferences

Version information for dms_tools and associated packages.
	Time and date: Tue Mar 27 18:58:17 2018
	Platform: Darwin-15.6.0-x86_64-i386-64bit
	Python version: 2.7.13 |Anaconda 4.4.0 (x86_64)| (default, Dec 20 2016, 23:05:08)  [GCC 4.2.1 Compatible Apple LLVM 6.0 (clang-600.0.57)]
	dms_tools version: 1.2.2
	Bio version: 1.70
	numpy version: 1.12.1
	scipy version: 0.19.0
	matplotlib version: 2.0.2
	cython version: 0.25.2
	pystan version: 2.16.0.0
	weblogolib version: 3.5.0
	PyPDF2 version: 1.26.0
	pandas version: 0.20.1

Parsed the following arguments:
	restrictdiffsel = None
	overlay3 = None
	colormap = jet
	nperline = 100
	numberevery = 10
	excludestop = False
	logoplot = HSF1i37vsDMSO37-rep1_diffsel_logoplot.pdf
	overlay_cmap = None
	mapmetric = charge
	diffprefheight = 1.0
	nosepline = False
	overlay1 = None
	overlay2 = None
	diffselheight = ['DMSO39vsDMSO37-rep1_mutdiffse

In [38]:
#rep2 HSF1i 37C vs DMSO 37C
!dms_logoplot HSF1i37vsDMSO37-rep2_mutdiffsel.txt HSF1i37vsDMSO37-rep2_diffsel_logoplot.pdf --nperline 100 --mapmetric charge --letterheight 2 --diffselheight DMSO39vsDMSO37-rep1_mutdiffsel.txt DMSO39vsDMSO37-rep2_mutdiffsel.txt DMSO39vsDMSO37-rep3_mutdiffsel.txt HSF1i37vsDMSO37-rep1_mutdiffsel.txt HSF1i37vsDMSO37-rep2_mutdiffsel.txt HSF1i37vsDMSO37-rep3_mutdiffsel.txt HSF1i39vsDMSO37-rep1_mutdiffsel.txt HSF1i39vsDMSO37-rep2_mutdiffsel.txt HSF1i39vsDMSO37-rep3_mutdiffsel.txt HSF1i39vsDMSO39-rep1_mutdiffsel.txt HSF1i39vsDMSO39-rep2_mutdiffsel.txt HSF1i39vsDMSO39-rep3_mutdiffsel.txt HSF1i39vsHSF1i37-rep1_mutdiffsel.txt HSF1i39vsHSF1i37-rep2_mutdiffsel.txt HSF1i39vsHSF1i37-rep3_mutdiffsel.txt Hsp90i37vsDMSO37-rep1_mutdiffsel.txt Hsp90i37vsDMSO37-rep2_mutdiffsel.txt Hsp90i39vsDMSO37-rep3_mutdiffsel.txt Hsp90i39vsDMSO37-rep1_mutdiffsel.txt Hsp90i39vsDMSO37-rep2_mutdiffsel.txt Hsp90i39vsDMSO37-rep3_mutdiffsel.txt  


Beginning execution of dms_logoplot in directory /Users/annapon/Documents/NP_DMS_2018/2018_NP_DMS/preferences

Version information for dms_tools and associated packages.
	Time and date: Tue Mar 27 18:58:37 2018
	Platform: Darwin-15.6.0-x86_64-i386-64bit
	Python version: 2.7.13 |Anaconda 4.4.0 (x86_64)| (default, Dec 20 2016, 23:05:08)  [GCC 4.2.1 Compatible Apple LLVM 6.0 (clang-600.0.57)]
	dms_tools version: 1.2.2
	Bio version: 1.70
	numpy version: 1.12.1
	scipy version: 0.19.0
	matplotlib version: 2.0.2
	cython version: 0.25.2
	pystan version: 2.16.0.0
	weblogolib version: 3.5.0
	PyPDF2 version: 1.26.0
	pandas version: 0.20.1

Parsed the following arguments:
	restrictdiffsel = None
	overlay3 = None
	colormap = jet
	nperline = 100
	numberevery = 10
	excludestop = False
	logoplot = HSF1i37vsDMSO37-rep2_diffsel_logoplot.pdf
	overlay_cmap = None
	mapmetric = charge
	diffprefheight = 1.0
	nosepline = False
	overlay1 = None
	overlay2 = None
	diffselheight = ['DMSO39vsDMSO37-rep1_mutdiffse

In [39]:
#rep3 HSF1i 37C vs DMSO 37C
!dms_logoplot HSF1i37vsDMSO37-rep3_mutdiffsel.txt HSF1i37vsDMSO37-rep3_diffsel_logoplot.pdf --nperline 100 --mapmetric charge --letterheight 2 --diffselheight DMSO39vsDMSO37-rep1_mutdiffsel.txt DMSO39vsDMSO37-rep2_mutdiffsel.txt DMSO39vsDMSO37-rep3_mutdiffsel.txt HSF1i37vsDMSO37-rep1_mutdiffsel.txt HSF1i37vsDMSO37-rep2_mutdiffsel.txt HSF1i37vsDMSO37-rep3_mutdiffsel.txt HSF1i39vsDMSO37-rep1_mutdiffsel.txt HSF1i39vsDMSO37-rep2_mutdiffsel.txt HSF1i39vsDMSO37-rep3_mutdiffsel.txt HSF1i39vsDMSO39-rep1_mutdiffsel.txt HSF1i39vsDMSO39-rep2_mutdiffsel.txt HSF1i39vsDMSO39-rep3_mutdiffsel.txt HSF1i39vsHSF1i37-rep1_mutdiffsel.txt HSF1i39vsHSF1i37-rep2_mutdiffsel.txt HSF1i39vsHSF1i37-rep3_mutdiffsel.txt Hsp90i37vsDMSO37-rep1_mutdiffsel.txt Hsp90i37vsDMSO37-rep2_mutdiffsel.txt Hsp90i39vsDMSO37-rep3_mutdiffsel.txt Hsp90i39vsDMSO37-rep1_mutdiffsel.txt Hsp90i39vsDMSO37-rep2_mutdiffsel.txt Hsp90i39vsDMSO37-rep3_mutdiffsel.txt  


Beginning execution of dms_logoplot in directory /Users/annapon/Documents/NP_DMS_2018/2018_NP_DMS/preferences

Version information for dms_tools and associated packages.
	Time and date: Tue Mar 27 18:58:55 2018
	Platform: Darwin-15.6.0-x86_64-i386-64bit
	Python version: 2.7.13 |Anaconda 4.4.0 (x86_64)| (default, Dec 20 2016, 23:05:08)  [GCC 4.2.1 Compatible Apple LLVM 6.0 (clang-600.0.57)]
	dms_tools version: 1.2.2
	Bio version: 1.70
	numpy version: 1.12.1
	scipy version: 0.19.0
	matplotlib version: 2.0.2
	cython version: 0.25.2
	pystan version: 2.16.0.0
	weblogolib version: 3.5.0
	PyPDF2 version: 1.26.0
	pandas version: 0.20.1

Parsed the following arguments:
	restrictdiffsel = None
	overlay3 = None
	colormap = jet
	nperline = 100
	numberevery = 10
	excludestop = False
	logoplot = HSF1i37vsDMSO37-rep3_diffsel_logoplot.pdf
	overlay_cmap = None
	mapmetric = charge
	diffprefheight = 1.0
	nosepline = False
	overlay1 = None
	overlay2 = None
	diffselheight = ['DMSO39vsDMSO37-rep1_mutdiffse

In [40]:
#rep1 HSF1i 39C vs DMSO 39C
!dms_logoplot HSF1i39vsDMSO39-rep1_mutdiffsel.txt HSF1i39vsDMSO39-rep1_diffsel_logoplot.pdf --nperline 100 --mapmetric charge --letterheight 2 --diffselheight DMSO39vsDMSO37-rep1_mutdiffsel.txt DMSO39vsDMSO37-rep2_mutdiffsel.txt DMSO39vsDMSO37-rep3_mutdiffsel.txt HSF1i37vsDMSO37-rep1_mutdiffsel.txt HSF1i37vsDMSO37-rep2_mutdiffsel.txt HSF1i37vsDMSO37-rep3_mutdiffsel.txt HSF1i39vsDMSO37-rep1_mutdiffsel.txt HSF1i39vsDMSO37-rep2_mutdiffsel.txt HSF1i39vsDMSO37-rep3_mutdiffsel.txt HSF1i39vsDMSO39-rep1_mutdiffsel.txt HSF1i39vsDMSO39-rep2_mutdiffsel.txt HSF1i39vsDMSO39-rep3_mutdiffsel.txt HSF1i39vsHSF1i37-rep1_mutdiffsel.txt HSF1i39vsHSF1i37-rep2_mutdiffsel.txt HSF1i39vsHSF1i37-rep3_mutdiffsel.txt Hsp90i37vsDMSO37-rep1_mutdiffsel.txt Hsp90i37vsDMSO37-rep2_mutdiffsel.txt Hsp90i39vsDMSO37-rep3_mutdiffsel.txt Hsp90i39vsDMSO37-rep1_mutdiffsel.txt Hsp90i39vsDMSO37-rep2_mutdiffsel.txt Hsp90i39vsDMSO37-rep3_mutdiffsel.txt  


Beginning execution of dms_logoplot in directory /Users/annapon/Documents/NP_DMS_2018/2018_NP_DMS/preferences

Version information for dms_tools and associated packages.
	Time and date: Tue Mar 27 18:59:14 2018
	Platform: Darwin-15.6.0-x86_64-i386-64bit
	Python version: 2.7.13 |Anaconda 4.4.0 (x86_64)| (default, Dec 20 2016, 23:05:08)  [GCC 4.2.1 Compatible Apple LLVM 6.0 (clang-600.0.57)]
	dms_tools version: 1.2.2
	Bio version: 1.70
	numpy version: 1.12.1
	scipy version: 0.19.0
	matplotlib version: 2.0.2
	cython version: 0.25.2
	pystan version: 2.16.0.0
	weblogolib version: 3.5.0
	PyPDF2 version: 1.26.0
	pandas version: 0.20.1

Parsed the following arguments:
	restrictdiffsel = None
	overlay3 = None
	colormap = jet
	nperline = 100
	numberevery = 10
	excludestop = False
	logoplot = HSF1i39vsDMSO39-rep1_diffsel_logoplot.pdf
	overlay_cmap = None
	mapmetric = charge
	diffprefheight = 1.0
	nosepline = False
	overlay1 = None
	overlay2 = None
	diffselheight = ['DMSO39vsDMSO37-rep1_mutdiffse

In [41]:
#rep2 HSF1i 39C vs DMSO 39C
!dms_logoplot HSF1i39vsDMSO39-rep2_mutdiffsel.txt HSF1i39vsDMSO39-rep2_diffsel_logoplot.pdf --nperline 100 --mapmetric charge --letterheight 2 --diffselheight DMSO39vsDMSO37-rep1_mutdiffsel.txt DMSO39vsDMSO37-rep2_mutdiffsel.txt DMSO39vsDMSO37-rep3_mutdiffsel.txt HSF1i37vsDMSO37-rep1_mutdiffsel.txt HSF1i37vsDMSO37-rep2_mutdiffsel.txt HSF1i37vsDMSO37-rep3_mutdiffsel.txt HSF1i39vsDMSO37-rep1_mutdiffsel.txt HSF1i39vsDMSO37-rep2_mutdiffsel.txt HSF1i39vsDMSO37-rep3_mutdiffsel.txt HSF1i39vsDMSO39-rep1_mutdiffsel.txt HSF1i39vsDMSO39-rep2_mutdiffsel.txt HSF1i39vsDMSO39-rep3_mutdiffsel.txt HSF1i39vsHSF1i37-rep1_mutdiffsel.txt HSF1i39vsHSF1i37-rep2_mutdiffsel.txt HSF1i39vsHSF1i37-rep3_mutdiffsel.txt Hsp90i37vsDMSO37-rep1_mutdiffsel.txt Hsp90i37vsDMSO37-rep2_mutdiffsel.txt Hsp90i39vsDMSO37-rep3_mutdiffsel.txt Hsp90i39vsDMSO37-rep1_mutdiffsel.txt Hsp90i39vsDMSO37-rep2_mutdiffsel.txt Hsp90i39vsDMSO37-rep3_mutdiffsel.txt  


Beginning execution of dms_logoplot in directory /Users/annapon/Documents/NP_DMS_2018/2018_NP_DMS/preferences

Version information for dms_tools and associated packages.
	Time and date: Tue Mar 27 18:59:35 2018
	Platform: Darwin-15.6.0-x86_64-i386-64bit
	Python version: 2.7.13 |Anaconda 4.4.0 (x86_64)| (default, Dec 20 2016, 23:05:08)  [GCC 4.2.1 Compatible Apple LLVM 6.0 (clang-600.0.57)]
	dms_tools version: 1.2.2
	Bio version: 1.70
	numpy version: 1.12.1
	scipy version: 0.19.0
	matplotlib version: 2.0.2
	cython version: 0.25.2
	pystan version: 2.16.0.0
	weblogolib version: 3.5.0
	PyPDF2 version: 1.26.0
	pandas version: 0.20.1

Parsed the following arguments:
	restrictdiffsel = None
	overlay3 = None
	colormap = jet
	nperline = 100
	numberevery = 10
	excludestop = False
	logoplot = HSF1i39vsDMSO39-rep2_diffsel_logoplot.pdf
	overlay_cmap = None
	mapmetric = charge
	diffprefheight = 1.0
	nosepline = False
	overlay1 = None
	overlay2 = None
	diffselheight = ['DMSO39vsDMSO37-rep1_mutdiffse

In [42]:
#rep3 HSF1i 39C vs DMSO 39C
!dms_logoplot HSF1i39vsDMSO39-rep3_mutdiffsel.txt HSF1i39vsDMSO39-rep3_diffsel_logoplot.pdf --nperline 100 --mapmetric charge --letterheight 2 --diffselheight DMSO39vsDMSO37-rep1_mutdiffsel.txt DMSO39vsDMSO37-rep2_mutdiffsel.txt DMSO39vsDMSO37-rep3_mutdiffsel.txt HSF1i37vsDMSO37-rep1_mutdiffsel.txt HSF1i37vsDMSO37-rep2_mutdiffsel.txt HSF1i37vsDMSO37-rep3_mutdiffsel.txt HSF1i39vsDMSO37-rep1_mutdiffsel.txt HSF1i39vsDMSO37-rep2_mutdiffsel.txt HSF1i39vsDMSO37-rep3_mutdiffsel.txt HSF1i39vsDMSO39-rep1_mutdiffsel.txt HSF1i39vsDMSO39-rep2_mutdiffsel.txt HSF1i39vsDMSO39-rep3_mutdiffsel.txt HSF1i39vsHSF1i37-rep1_mutdiffsel.txt HSF1i39vsHSF1i37-rep2_mutdiffsel.txt HSF1i39vsHSF1i37-rep3_mutdiffsel.txt Hsp90i37vsDMSO37-rep1_mutdiffsel.txt Hsp90i37vsDMSO37-rep2_mutdiffsel.txt Hsp90i39vsDMSO37-rep3_mutdiffsel.txt Hsp90i39vsDMSO37-rep1_mutdiffsel.txt Hsp90i39vsDMSO37-rep2_mutdiffsel.txt Hsp90i39vsDMSO37-rep3_mutdiffsel.txt  


Beginning execution of dms_logoplot in directory /Users/annapon/Documents/NP_DMS_2018/2018_NP_DMS/preferences

Version information for dms_tools and associated packages.
	Time and date: Tue Mar 27 19:00:04 2018
	Platform: Darwin-15.6.0-x86_64-i386-64bit
	Python version: 2.7.13 |Anaconda 4.4.0 (x86_64)| (default, Dec 20 2016, 23:05:08)  [GCC 4.2.1 Compatible Apple LLVM 6.0 (clang-600.0.57)]
	dms_tools version: 1.2.2
	Bio version: 1.70
	numpy version: 1.12.1
	scipy version: 0.19.0
	matplotlib version: 2.0.2
	cython version: 0.25.2
	pystan version: 2.16.0.0
	weblogolib version: 3.5.0
	PyPDF2 version: 1.26.0
	pandas version: 0.20.1

Parsed the following arguments:
	restrictdiffsel = None
	overlay3 = None
	colormap = jet
	nperline = 100
	numberevery = 10
	excludestop = False
	logoplot = HSF1i39vsDMSO39-rep3_diffsel_logoplot.pdf
	overlay_cmap = None
	mapmetric = charge
	diffprefheight = 1.0
	nosepline = False
	overlay1 = None
	overlay2 = None
	diffselheight = ['DMSO39vsDMSO37-rep1_mutdiffse

In [43]:
#rep1 HSF1i 39C vs HSF1i 37C
!dms_logoplot HSF1i39vsHSF1i37-rep1_mutdiffsel.txt HSF1i39vsHSF1i37-rep1_diffsel_logoplot.pdf --nperline 100 --mapmetric charge --letterheight 2 --diffselheight DMSO39vsDMSO37-rep1_mutdiffsel.txt DMSO39vsDMSO37-rep2_mutdiffsel.txt DMSO39vsDMSO37-rep3_mutdiffsel.txt HSF1i37vsDMSO37-rep1_mutdiffsel.txt HSF1i37vsDMSO37-rep2_mutdiffsel.txt HSF1i37vsDMSO37-rep3_mutdiffsel.txt HSF1i39vsDMSO37-rep1_mutdiffsel.txt HSF1i39vsDMSO37-rep2_mutdiffsel.txt HSF1i39vsDMSO37-rep3_mutdiffsel.txt HSF1i39vsDMSO39-rep1_mutdiffsel.txt HSF1i39vsDMSO39-rep2_mutdiffsel.txt HSF1i39vsDMSO39-rep3_mutdiffsel.txt HSF1i39vsHSF1i37-rep1_mutdiffsel.txt HSF1i39vsHSF1i37-rep2_mutdiffsel.txt HSF1i39vsHSF1i37-rep3_mutdiffsel.txt Hsp90i37vsDMSO37-rep1_mutdiffsel.txt Hsp90i37vsDMSO37-rep2_mutdiffsel.txt Hsp90i39vsDMSO37-rep3_mutdiffsel.txt Hsp90i39vsDMSO37-rep1_mutdiffsel.txt Hsp90i39vsDMSO37-rep2_mutdiffsel.txt Hsp90i39vsDMSO37-rep3_mutdiffsel.txt  


Beginning execution of dms_logoplot in directory /Users/annapon/Documents/NP_DMS_2018/2018_NP_DMS/preferences

Version information for dms_tools and associated packages.
	Time and date: Tue Mar 27 19:00:21 2018
	Platform: Darwin-15.6.0-x86_64-i386-64bit
	Python version: 2.7.13 |Anaconda 4.4.0 (x86_64)| (default, Dec 20 2016, 23:05:08)  [GCC 4.2.1 Compatible Apple LLVM 6.0 (clang-600.0.57)]
	dms_tools version: 1.2.2
	Bio version: 1.70
	numpy version: 1.12.1
	scipy version: 0.19.0
	matplotlib version: 2.0.2
	cython version: 0.25.2
	pystan version: 2.16.0.0
	weblogolib version: 3.5.0
	PyPDF2 version: 1.26.0
	pandas version: 0.20.1

Parsed the following arguments:
	restrictdiffsel = None
	overlay3 = None
	colormap = jet
	nperline = 100
	numberevery = 10
	excludestop = False
	logoplot = HSF1i39vsHSF1i37-rep1_diffsel_logoplot.pdf
	overlay_cmap = None
	mapmetric = charge
	diffprefheight = 1.0
	nosepline = False
	overlay1 = None
	overlay2 = None
	diffselheight = ['DMSO39vsDMSO37-rep1_mutdiffs

In [44]:
#rep2 HSF1i 39C vs HSF1i 37C
!dms_logoplot HSF1i39vsHSF1i37-rep2_mutdiffsel.txt HSF1i39vsHSF1i37-rep2_diffsel_logoplot.pdf --nperline 100 --mapmetric charge --letterheight 2 --diffselheight DMSO39vsDMSO37-rep1_mutdiffsel.txt DMSO39vsDMSO37-rep2_mutdiffsel.txt DMSO39vsDMSO37-rep3_mutdiffsel.txt HSF1i37vsDMSO37-rep1_mutdiffsel.txt HSF1i37vsDMSO37-rep2_mutdiffsel.txt HSF1i37vsDMSO37-rep3_mutdiffsel.txt HSF1i39vsDMSO37-rep1_mutdiffsel.txt HSF1i39vsDMSO37-rep2_mutdiffsel.txt HSF1i39vsDMSO37-rep3_mutdiffsel.txt HSF1i39vsDMSO39-rep1_mutdiffsel.txt HSF1i39vsDMSO39-rep2_mutdiffsel.txt HSF1i39vsDMSO39-rep3_mutdiffsel.txt HSF1i39vsHSF1i37-rep1_mutdiffsel.txt HSF1i39vsHSF1i37-rep2_mutdiffsel.txt HSF1i39vsHSF1i37-rep3_mutdiffsel.txt Hsp90i37vsDMSO37-rep1_mutdiffsel.txt Hsp90i37vsDMSO37-rep2_mutdiffsel.txt Hsp90i39vsDMSO37-rep3_mutdiffsel.txt Hsp90i39vsDMSO37-rep1_mutdiffsel.txt Hsp90i39vsDMSO37-rep2_mutdiffsel.txt Hsp90i39vsDMSO37-rep3_mutdiffsel.txt  


Beginning execution of dms_logoplot in directory /Users/annapon/Documents/NP_DMS_2018/2018_NP_DMS/preferences

Version information for dms_tools and associated packages.
	Time and date: Tue Mar 27 19:00:39 2018
	Platform: Darwin-15.6.0-x86_64-i386-64bit
	Python version: 2.7.13 |Anaconda 4.4.0 (x86_64)| (default, Dec 20 2016, 23:05:08)  [GCC 4.2.1 Compatible Apple LLVM 6.0 (clang-600.0.57)]
	dms_tools version: 1.2.2
	Bio version: 1.70
	numpy version: 1.12.1
	scipy version: 0.19.0
	matplotlib version: 2.0.2
	cython version: 0.25.2
	pystan version: 2.16.0.0
	weblogolib version: 3.5.0
	PyPDF2 version: 1.26.0
	pandas version: 0.20.1

Parsed the following arguments:
	restrictdiffsel = None
	overlay3 = None
	colormap = jet
	nperline = 100
	numberevery = 10
	excludestop = False
	logoplot = HSF1i39vsHSF1i37-rep2_diffsel_logoplot.pdf
	overlay_cmap = None
	mapmetric = charge
	diffprefheight = 1.0
	nosepline = False
	overlay1 = None
	overlay2 = None
	diffselheight = ['DMSO39vsDMSO37-rep1_mutdiffs

In [45]:
#rep3 HSF1i 39C vs HSF1i 37C
!dms_logoplot HSF1i39vsHSF1i37-rep3_mutdiffsel.txt HSF1i39vsHSF1i37-rep3_diffsel_logoplot.pdf --nperline 100 --mapmetric charge --letterheight 2 --diffselheight DMSO39vsDMSO37-rep1_mutdiffsel.txt DMSO39vsDMSO37-rep2_mutdiffsel.txt DMSO39vsDMSO37-rep3_mutdiffsel.txt HSF1i37vsDMSO37-rep1_mutdiffsel.txt HSF1i37vsDMSO37-rep2_mutdiffsel.txt HSF1i37vsDMSO37-rep3_mutdiffsel.txt HSF1i39vsDMSO37-rep1_mutdiffsel.txt HSF1i39vsDMSO37-rep2_mutdiffsel.txt HSF1i39vsDMSO37-rep3_mutdiffsel.txt HSF1i39vsDMSO39-rep1_mutdiffsel.txt HSF1i39vsDMSO39-rep2_mutdiffsel.txt HSF1i39vsDMSO39-rep3_mutdiffsel.txt HSF1i39vsHSF1i37-rep1_mutdiffsel.txt HSF1i39vsHSF1i37-rep2_mutdiffsel.txt HSF1i39vsHSF1i37-rep3_mutdiffsel.txt Hsp90i37vsDMSO37-rep1_mutdiffsel.txt Hsp90i37vsDMSO37-rep2_mutdiffsel.txt Hsp90i39vsDMSO37-rep3_mutdiffsel.txt Hsp90i39vsDMSO37-rep1_mutdiffsel.txt Hsp90i39vsDMSO37-rep2_mutdiffsel.txt Hsp90i39vsDMSO37-rep3_mutdiffsel.txt  


Beginning execution of dms_logoplot in directory /Users/annapon/Documents/NP_DMS_2018/2018_NP_DMS/preferences

Version information for dms_tools and associated packages.
	Time and date: Tue Mar 27 19:00:57 2018
	Platform: Darwin-15.6.0-x86_64-i386-64bit
	Python version: 2.7.13 |Anaconda 4.4.0 (x86_64)| (default, Dec 20 2016, 23:05:08)  [GCC 4.2.1 Compatible Apple LLVM 6.0 (clang-600.0.57)]
	dms_tools version: 1.2.2
	Bio version: 1.70
	numpy version: 1.12.1
	scipy version: 0.19.0
	matplotlib version: 2.0.2
	cython version: 0.25.2
	pystan version: 2.16.0.0
	weblogolib version: 3.5.0
	PyPDF2 version: 1.26.0
	pandas version: 0.20.1

Parsed the following arguments:
	restrictdiffsel = None
	overlay3 = None
	colormap = jet
	nperline = 100
	numberevery = 10
	excludestop = False
	logoplot = HSF1i39vsHSF1i37-rep3_diffsel_logoplot.pdf
	overlay_cmap = None
	mapmetric = charge
	diffprefheight = 1.0
	nosepline = False
	overlay1 = None
	overlay2 = None
	diffselheight = ['DMSO39vsDMSO37-rep1_mutdiffs

In [46]:
#rep1 Hsp90i 37C vs DMSO 37C
!dms_logoplot Hsp90i37vsDMSO37-rep1_mutdiffsel.txt Hsp90i37vsDMSO37-rep1_diffsel_logoplot.pdf --nperline 100 --mapmetric charge --letterheight 2 --diffselheight DMSO39vsDMSO37-rep1_mutdiffsel.txt DMSO39vsDMSO37-rep2_mutdiffsel.txt DMSO39vsDMSO37-rep3_mutdiffsel.txt HSF1i37vsDMSO37-rep1_mutdiffsel.txt HSF1i37vsDMSO37-rep2_mutdiffsel.txt HSF1i37vsDMSO37-rep3_mutdiffsel.txt HSF1i39vsDMSO37-rep1_mutdiffsel.txt HSF1i39vsDMSO37-rep2_mutdiffsel.txt HSF1i39vsDMSO37-rep3_mutdiffsel.txt HSF1i39vsDMSO39-rep1_mutdiffsel.txt HSF1i39vsDMSO39-rep2_mutdiffsel.txt HSF1i39vsDMSO39-rep3_mutdiffsel.txt HSF1i39vsHSF1i37-rep1_mutdiffsel.txt HSF1i39vsHSF1i37-rep2_mutdiffsel.txt HSF1i39vsHSF1i37-rep3_mutdiffsel.txt Hsp90i37vsDMSO37-rep1_mutdiffsel.txt Hsp90i37vsDMSO37-rep2_mutdiffsel.txt Hsp90i39vsDMSO37-rep3_mutdiffsel.txt Hsp90i39vsDMSO37-rep1_mutdiffsel.txt Hsp90i39vsDMSO37-rep2_mutdiffsel.txt Hsp90i39vsDMSO37-rep3_mutdiffsel.txt  


Beginning execution of dms_logoplot in directory /Users/annapon/Documents/NP_DMS_2018/2018_NP_DMS/preferences

Version information for dms_tools and associated packages.
	Time and date: Tue Mar 27 19:01:15 2018
	Platform: Darwin-15.6.0-x86_64-i386-64bit
	Python version: 2.7.13 |Anaconda 4.4.0 (x86_64)| (default, Dec 20 2016, 23:05:08)  [GCC 4.2.1 Compatible Apple LLVM 6.0 (clang-600.0.57)]
	dms_tools version: 1.2.2
	Bio version: 1.70
	numpy version: 1.12.1
	scipy version: 0.19.0
	matplotlib version: 2.0.2
	cython version: 0.25.2
	pystan version: 2.16.0.0
	weblogolib version: 3.5.0
	PyPDF2 version: 1.26.0
	pandas version: 0.20.1

Parsed the following arguments:
	restrictdiffsel = None
	overlay3 = None
	colormap = jet
	nperline = 100
	numberevery = 10
	excludestop = False
	logoplot = Hsp90i37vsDMSO37-rep1_diffsel_logoplot.pdf
	overlay_cmap = None
	mapmetric = charge
	diffprefheight = 1.0
	nosepline = False
	overlay1 = None
	overlay2 = None
	diffselheight = ['DMSO39vsDMSO37-rep1_mutdiffs

In [47]:
#rep2 Hsp90i 37C vs DMSO 37C
!dms_logoplot Hsp90i37vsDMSO37-rep2_mutdiffsel.txt Hsp90i37vsDMSO37-rep2_diffsel_logoplot.pdf --nperline 100 --mapmetric charge --letterheight 2 --diffselheight DMSO39vsDMSO37-rep1_mutdiffsel.txt DMSO39vsDMSO37-rep2_mutdiffsel.txt DMSO39vsDMSO37-rep3_mutdiffsel.txt HSF1i37vsDMSO37-rep1_mutdiffsel.txt HSF1i37vsDMSO37-rep2_mutdiffsel.txt HSF1i37vsDMSO37-rep3_mutdiffsel.txt HSF1i39vsDMSO37-rep1_mutdiffsel.txt HSF1i39vsDMSO37-rep2_mutdiffsel.txt HSF1i39vsDMSO37-rep3_mutdiffsel.txt HSF1i39vsDMSO39-rep1_mutdiffsel.txt HSF1i39vsDMSO39-rep2_mutdiffsel.txt HSF1i39vsDMSO39-rep3_mutdiffsel.txt HSF1i39vsHSF1i37-rep1_mutdiffsel.txt HSF1i39vsHSF1i37-rep2_mutdiffsel.txt HSF1i39vsHSF1i37-rep3_mutdiffsel.txt Hsp90i37vsDMSO37-rep1_mutdiffsel.txt Hsp90i37vsDMSO37-rep2_mutdiffsel.txt Hsp90i39vsDMSO37-rep3_mutdiffsel.txt Hsp90i39vsDMSO37-rep1_mutdiffsel.txt Hsp90i39vsDMSO37-rep2_mutdiffsel.txt Hsp90i39vsDMSO37-rep3_mutdiffsel.txt  


Beginning execution of dms_logoplot in directory /Users/annapon/Documents/NP_DMS_2018/2018_NP_DMS/preferences

Version information for dms_tools and associated packages.
	Time and date: Tue Mar 27 19:01:33 2018
	Platform: Darwin-15.6.0-x86_64-i386-64bit
	Python version: 2.7.13 |Anaconda 4.4.0 (x86_64)| (default, Dec 20 2016, 23:05:08)  [GCC 4.2.1 Compatible Apple LLVM 6.0 (clang-600.0.57)]
	dms_tools version: 1.2.2
	Bio version: 1.70
	numpy version: 1.12.1
	scipy version: 0.19.0
	matplotlib version: 2.0.2
	cython version: 0.25.2
	pystan version: 2.16.0.0
	weblogolib version: 3.5.0
	PyPDF2 version: 1.26.0
	pandas version: 0.20.1

Parsed the following arguments:
	restrictdiffsel = None
	overlay3 = None
	colormap = jet
	nperline = 100
	numberevery = 10
	excludestop = False
	logoplot = Hsp90i37vsDMSO37-rep2_diffsel_logoplot.pdf
	overlay_cmap = None
	mapmetric = charge
	diffprefheight = 1.0
	nosepline = False
	overlay1 = None
	overlay2 = None
	diffselheight = ['DMSO39vsDMSO37-rep1_mutdiffs

In [50]:
#rep3 Hsp90i 37C vs DMSO 37C
!dms_logoplot Hsp90i39vsDMSO37-rep3_mutdiffsel.txt Hsp90i37vsDMSO37-rep3_diffsel_logoplot.pdf --nperline 100 --mapmetric charge --letterheight 2 --diffselheight DMSO39vsDMSO37-rep1_mutdiffsel.txt DMSO39vsDMSO37-rep2_mutdiffsel.txt DMSO39vsDMSO37-rep3_mutdiffsel.txt HSF1i37vsDMSO37-rep1_mutdiffsel.txt HSF1i37vsDMSO37-rep2_mutdiffsel.txt HSF1i37vsDMSO37-rep3_mutdiffsel.txt HSF1i39vsDMSO37-rep1_mutdiffsel.txt HSF1i39vsDMSO37-rep2_mutdiffsel.txt HSF1i39vsDMSO37-rep3_mutdiffsel.txt HSF1i39vsDMSO39-rep1_mutdiffsel.txt HSF1i39vsDMSO39-rep2_mutdiffsel.txt HSF1i39vsDMSO39-rep3_mutdiffsel.txt HSF1i39vsHSF1i37-rep1_mutdiffsel.txt HSF1i39vsHSF1i37-rep2_mutdiffsel.txt HSF1i39vsHSF1i37-rep3_mutdiffsel.txt Hsp90i37vsDMSO37-rep1_mutdiffsel.txt Hsp90i37vsDMSO37-rep2_mutdiffsel.txt Hsp90i39vsDMSO37-rep3_mutdiffsel.txt Hsp90i39vsDMSO37-rep1_mutdiffsel.txt Hsp90i39vsDMSO37-rep2_mutdiffsel.txt Hsp90i39vsDMSO37-rep3_mutdiffsel.txt  


Beginning execution of dms_logoplot in directory /Users/annapon/Documents/NP_DMS_2018/2018_NP_DMS/preferences

Version information for dms_tools and associated packages.
	Time and date: Tue Mar 27 19:02:46 2018
	Platform: Darwin-15.6.0-x86_64-i386-64bit
	Python version: 2.7.13 |Anaconda 4.4.0 (x86_64)| (default, Dec 20 2016, 23:05:08)  [GCC 4.2.1 Compatible Apple LLVM 6.0 (clang-600.0.57)]
	dms_tools version: 1.2.2
	Bio version: 1.70
	numpy version: 1.12.1
	scipy version: 0.19.0
	matplotlib version: 2.0.2
	cython version: 0.25.2
	pystan version: 2.16.0.0
	weblogolib version: 3.5.0
	PyPDF2 version: 1.26.0
	pandas version: 0.20.1

Parsed the following arguments:
	restrictdiffsel = None
	overlay3 = None
	colormap = jet
	nperline = 100
	numberevery = 10
	excludestop = False
	logoplot = Hsp90i37vsDMSO37-rep3_diffsel_logoplot.pdf
	overlay_cmap = None
	mapmetric = charge
	diffprefheight = 1.0
	nosepline = False
	overlay1 = None
	overlay2 = None
	diffselheight = ['DMSO39vsDMSO37-rep1_mutdiffs

In [51]:
#rep1 Hsp90i 39C vs DMSO 37C
!dms_logoplot Hsp90i39vsDMSO37-rep1_mutdiffsel.txt Hsp90i39vsDMSO37-rep1_diffsel_logoplot.pdf --nperline 100 --mapmetric charge --letterheight 2 --diffselheight DMSO39vsDMSO37-rep1_mutdiffsel.txt DMSO39vsDMSO37-rep2_mutdiffsel.txt DMSO39vsDMSO37-rep3_mutdiffsel.txt HSF1i37vsDMSO37-rep1_mutdiffsel.txt HSF1i37vsDMSO37-rep2_mutdiffsel.txt HSF1i37vsDMSO37-rep3_mutdiffsel.txt HSF1i39vsDMSO37-rep1_mutdiffsel.txt HSF1i39vsDMSO37-rep2_mutdiffsel.txt HSF1i39vsDMSO37-rep3_mutdiffsel.txt HSF1i39vsDMSO39-rep1_mutdiffsel.txt HSF1i39vsDMSO39-rep2_mutdiffsel.txt HSF1i39vsDMSO39-rep3_mutdiffsel.txt HSF1i39vsHSF1i37-rep1_mutdiffsel.txt HSF1i39vsHSF1i37-rep2_mutdiffsel.txt HSF1i39vsHSF1i37-rep3_mutdiffsel.txt Hsp90i37vsDMSO37-rep1_mutdiffsel.txt Hsp90i37vsDMSO37-rep2_mutdiffsel.txt Hsp90i39vsDMSO37-rep3_mutdiffsel.txt Hsp90i39vsDMSO37-rep1_mutdiffsel.txt Hsp90i39vsDMSO37-rep2_mutdiffsel.txt Hsp90i39vsDMSO37-rep3_mutdiffsel.txt  


Beginning execution of dms_logoplot in directory /Users/annapon/Documents/NP_DMS_2018/2018_NP_DMS/preferences

Version information for dms_tools and associated packages.
	Time and date: Tue Mar 27 19:03:05 2018
	Platform: Darwin-15.6.0-x86_64-i386-64bit
	Python version: 2.7.13 |Anaconda 4.4.0 (x86_64)| (default, Dec 20 2016, 23:05:08)  [GCC 4.2.1 Compatible Apple LLVM 6.0 (clang-600.0.57)]
	dms_tools version: 1.2.2
	Bio version: 1.70
	numpy version: 1.12.1
	scipy version: 0.19.0
	matplotlib version: 2.0.2
	cython version: 0.25.2
	pystan version: 2.16.0.0
	weblogolib version: 3.5.0
	PyPDF2 version: 1.26.0
	pandas version: 0.20.1

Parsed the following arguments:
	restrictdiffsel = None
	overlay3 = None
	colormap = jet
	nperline = 100
	numberevery = 10
	excludestop = False
	logoplot = Hsp90i39vsDMSO37-rep1_diffsel_logoplot.pdf
	overlay_cmap = None
	mapmetric = charge
	diffprefheight = 1.0
	nosepline = False
	overlay1 = None
	overlay2 = None
	diffselheight = ['DMSO39vsDMSO37-rep1_mutdiffs

In [52]:
#rep2 Hsp90i 39C vs DMSO 37C
!dms_logoplot Hsp90i39vsDMSO37-rep2_mutdiffsel.txt Hsp90i39vsDMSO37-rep2_diffsel_logoplot.pdf --nperline 100 --mapmetric charge --letterheight 2 --diffselheight DMSO39vsDMSO37-rep1_mutdiffsel.txt DMSO39vsDMSO37-rep2_mutdiffsel.txt DMSO39vsDMSO37-rep3_mutdiffsel.txt HSF1i37vsDMSO37-rep1_mutdiffsel.txt HSF1i37vsDMSO37-rep2_mutdiffsel.txt HSF1i37vsDMSO37-rep3_mutdiffsel.txt HSF1i39vsDMSO37-rep1_mutdiffsel.txt HSF1i39vsDMSO37-rep2_mutdiffsel.txt HSF1i39vsDMSO37-rep3_mutdiffsel.txt HSF1i39vsDMSO39-rep1_mutdiffsel.txt HSF1i39vsDMSO39-rep2_mutdiffsel.txt HSF1i39vsDMSO39-rep3_mutdiffsel.txt HSF1i39vsHSF1i37-rep1_mutdiffsel.txt HSF1i39vsHSF1i37-rep2_mutdiffsel.txt HSF1i39vsHSF1i37-rep3_mutdiffsel.txt Hsp90i37vsDMSO37-rep1_mutdiffsel.txt Hsp90i37vsDMSO37-rep2_mutdiffsel.txt Hsp90i39vsDMSO37-rep3_mutdiffsel.txt Hsp90i39vsDMSO37-rep1_mutdiffsel.txt Hsp90i39vsDMSO37-rep2_mutdiffsel.txt Hsp90i39vsDMSO37-rep3_mutdiffsel.txt  


Beginning execution of dms_logoplot in directory /Users/annapon/Documents/NP_DMS_2018/2018_NP_DMS/preferences

Version information for dms_tools and associated packages.
	Time and date: Tue Mar 27 19:03:23 2018
	Platform: Darwin-15.6.0-x86_64-i386-64bit
	Python version: 2.7.13 |Anaconda 4.4.0 (x86_64)| (default, Dec 20 2016, 23:05:08)  [GCC 4.2.1 Compatible Apple LLVM 6.0 (clang-600.0.57)]
	dms_tools version: 1.2.2
	Bio version: 1.70
	numpy version: 1.12.1
	scipy version: 0.19.0
	matplotlib version: 2.0.2
	cython version: 0.25.2
	pystan version: 2.16.0.0
	weblogolib version: 3.5.0
	PyPDF2 version: 1.26.0
	pandas version: 0.20.1

Parsed the following arguments:
	restrictdiffsel = None
	overlay3 = None
	colormap = jet
	nperline = 100
	numberevery = 10
	excludestop = False
	logoplot = Hsp90i39vsDMSO37-rep2_diffsel_logoplot.pdf
	overlay_cmap = None
	mapmetric = charge
	diffprefheight = 1.0
	nosepline = False
	overlay1 = None
	overlay2 = None
	diffselheight = ['DMSO39vsDMSO37-rep1_mutdiffs

In [53]:
#rep3 Hsp90i 39C vs DMSO 37C
!dms_logoplot Hsp90i39vsDMSO37-rep3_mutdiffsel.txt Hsp90i39vsDMSO37-rep3_diffsel_logoplot.pdf --nperline 100 --mapmetric charge --letterheight 2 --diffselheight DMSO39vsDMSO37-rep1_mutdiffsel.txt DMSO39vsDMSO37-rep2_mutdiffsel.txt DMSO39vsDMSO37-rep3_mutdiffsel.txt HSF1i37vsDMSO37-rep1_mutdiffsel.txt HSF1i37vsDMSO37-rep2_mutdiffsel.txt HSF1i37vsDMSO37-rep3_mutdiffsel.txt HSF1i39vsDMSO37-rep1_mutdiffsel.txt HSF1i39vsDMSO37-rep2_mutdiffsel.txt HSF1i39vsDMSO37-rep3_mutdiffsel.txt HSF1i39vsDMSO39-rep1_mutdiffsel.txt HSF1i39vsDMSO39-rep2_mutdiffsel.txt HSF1i39vsDMSO39-rep3_mutdiffsel.txt HSF1i39vsHSF1i37-rep1_mutdiffsel.txt HSF1i39vsHSF1i37-rep2_mutdiffsel.txt HSF1i39vsHSF1i37-rep3_mutdiffsel.txt Hsp90i37vsDMSO37-rep1_mutdiffsel.txt Hsp90i37vsDMSO37-rep2_mutdiffsel.txt Hsp90i39vsDMSO37-rep3_mutdiffsel.txt Hsp90i39vsDMSO37-rep1_mutdiffsel.txt Hsp90i39vsDMSO37-rep2_mutdiffsel.txt Hsp90i39vsDMSO37-rep3_mutdiffsel.txt  


Beginning execution of dms_logoplot in directory /Users/annapon/Documents/NP_DMS_2018/2018_NP_DMS/preferences

Version information for dms_tools and associated packages.
	Time and date: Tue Mar 27 19:03:41 2018
	Platform: Darwin-15.6.0-x86_64-i386-64bit
	Python version: 2.7.13 |Anaconda 4.4.0 (x86_64)| (default, Dec 20 2016, 23:05:08)  [GCC 4.2.1 Compatible Apple LLVM 6.0 (clang-600.0.57)]
	dms_tools version: 1.2.2
	Bio version: 1.70
	numpy version: 1.12.1
	scipy version: 0.19.0
	matplotlib version: 2.0.2
	cython version: 0.25.2
	pystan version: 2.16.0.0
	weblogolib version: 3.5.0
	PyPDF2 version: 1.26.0
	pandas version: 0.20.1

Parsed the following arguments:
	restrictdiffsel = None
	overlay3 = None
	colormap = jet
	nperline = 100
	numberevery = 10
	excludestop = False
	logoplot = Hsp90i39vsDMSO37-rep3_diffsel_logoplot.pdf
	overlay_cmap = None
	mapmetric = charge
	diffprefheight = 1.0
	nosepline = False
	overlay1 = None
	overlay2 = None
	diffselheight = ['DMSO39vsDMSO37-rep1_mutdiffs

### Generate logo plot with MxA selection

Using the data from O. Ashenberg et al 2017 to compare with the differential selection observed in the present work.

In [11]:
#MxA logo plot with height scaled to perturbed proteostasis selection
%cd preferences
!dms_logoplot MxA.txt MxA_logoplot.pdf --nperline 100 --mapmetric charge --letterheight 2 --diffselheight DMSO39vsDMSO37-rep1_mutdiffsel.txt DMSO39vsDMSO37-rep2_mutdiffsel.txt DMSO39vsDMSO37-rep3_mutdiffsel.txt HSF1i37vsDMSO37-rep1_mutdiffsel.txt HSF1i37vsDMSO37-rep2_mutdiffsel.txt HSF1i37vsDMSO37-rep3_mutdiffsel.txt HSF1i39vsDMSO37-rep1_mutdiffsel.txt HSF1i39vsDMSO37-rep2_mutdiffsel.txt HSF1i39vsDMSO37-rep3_mutdiffsel.txt HSF1i39vsDMSO39-rep1_mutdiffsel.txt HSF1i39vsDMSO39-rep2_mutdiffsel.txt HSF1i39vsDMSO39-rep3_mutdiffsel.txt HSF1i39vsHSF1i37-rep1_mutdiffsel.txt HSF1i39vsHSF1i37-rep2_mutdiffsel.txt HSF1i39vsHSF1i37-rep3_mutdiffsel.txt Hsp90i37vsDMSO37-rep1_mutdiffsel.txt Hsp90i37vsDMSO37-rep2_mutdiffsel.txt Hsp90i39vsDMSO37-rep3_mutdiffsel.txt Hsp90i39vsDMSO37-rep1_mutdiffsel.txt Hsp90i39vsDMSO37-rep2_mutdiffsel.txt Hsp90i39vsDMSO37-rep3_mutdiffsel.txt MxA.txt 

/Users/annapon/Documents/NP_DMS_2018/2018_NP_DMS/preferences

Beginning execution of dms_logoplot in directory /Users/annapon/Documents/NP_DMS_2018/2018_NP_DMS/preferences

Version information for dms_tools and associated packages.
	Time and date: Wed Mar 28 15:39:25 2018
	Platform: Darwin-15.6.0-x86_64-i386-64bit
	Python version: 2.7.13 |Anaconda 4.4.0 (x86_64)| (default, Dec 20 2016, 23:05:08)  [GCC 4.2.1 Compatible Apple LLVM 6.0 (clang-600.0.57)]
	dms_tools version: 1.2.2
	Bio version: 1.70
	numpy version: 1.12.1
	scipy version: 0.19.0
	matplotlib version: 2.0.2
	cython version: 0.25.2
	pystan version: 2.16.0.0
	weblogolib version: 3.5.0
	PyPDF2 version: 1.26.0
	pandas version: 0.20.1

Parsed the following arguments:
	restrictdiffsel = None
	overlay3 = None
	colormap = jet
	nperline = 100
	numberevery = 10
	excludestop = False
	logoplot = MxA_logoplot.pdf
	overlay_cmap = None
	mapmetric = charge
	diffprefheight = 1.0
	nosepline = False
	overlay1 = None
	overlay2 = None
	diffselheig